# Sentiment Analysis on Short-Term Rental Reviews

## Part 1: Loading Data into Python Dataframe

In [1]:
# In starting out with the product developement, the first step will be getting the data into a dataframe.
# This will allow me to pre-process the data with Pandas built in functions [1].
# In order to do this, pandas must be added to the notebook. 
import pandas as pd

In [2]:
# next, I will read in the CSV into a pandas dataframe
file = "Hotel_Reviews.csv"
original_data = pd.read_csv(file)

In [3]:
#show some of the data to start to understand it better. 
original_data.head(3)

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968


In [4]:
# for sentiment analysis, we will want to combine the data into a new dataframe that will be used for pre-processing.
# the fields that are relevant are the positive review, negative review, and the reviewer score
# reviewer score can be used for labelling our sentiment as per the review 

original_data["Total_Review"] = original_data["Negative_Review"] + " " + original_data["Positive_Review"]

columns = ["Total_Review", "Reviewer_Score"]

review_data = original_data[columns]

review_data.head(3)

,Total_Review,Reviewer_Score
0,I am so angry that i made this post available...,2.9
1,No Negative No real complaints the hotel was ...,7.5
2,Rooms are nice but for elderly a bit difficul...,7.1


## Part 3: Optimizing the Pre-Processing into one Function

This section will be used for developing final preprocessing pipelines that can be used in my ML development. There will be two because one will be with using POS tagging and the other will be without.

In [5]:
#import the relevant libraries and download the relevant nltk dependancies [3]
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

#download the needed nltk toolkits
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

# optimized all-encompassing function for preprocessing that can easily be manipulated for tests
def preprocess_pos(review):
    
    #preprocess remove special characters and ensure lowercase 
    def preprocess_remove_special(review):
        review_handled = re.sub(r'[^a-zA-Z\s]', '', review)
        review_lower = review_handled.lower()
        return review_lower

    #tokenize the words
    def preprocess_tokenize(review):
        bag_of_words = word_tokenize(review)
        return bag_of_words

    #remove the stopwords
    def preprocess_stopwords(review):
        words = stopwords.words('english')
        set_stop_words = set(words)
        removed_stopwords_words = []
        for word in review:
            if word not in set_stop_words:
                removed_stopwords_words.append(word)
        return removed_stopwords_words

    #lemmatize
    def preprocess_lemmatize(review):
        lemmatizer = WordNetLemmatizer()
        lemmatized_review = [lemmatizer.lemmatize(word) for word in review ]
        return lemmatized_review

    #POS Tagging
    def preprocess_tagger(review):
        tagged_review = nltk.pos_tag(review)
        return tagged_review


    #execute the individual part functions within this bigger function
    review_data_copy = review_data.copy()
    review_data_copy['Total_Review_handled'] = review_data_copy["Total_Review"].apply(preprocess_remove_special)
    review_data_copy['Total_Review_handled'] = review_data_copy['Total_Review_handled'].apply(preprocess_tokenize)
    review_data_copy['Total_Review_handled'] = review_data_copy['Total_Review_handled'].apply(preprocess_stopwords)
    review_data_copy['Total_Review_handled'] = review_data_copy['Total_Review_handled'].apply(preprocess_lemmatize)
    review_data_copy['POS_Tag'] = review_data_copy['Total_Review_handled'].apply(preprocess_tagger)

    #create the final data
    review_data_final = review_data_copy[['POS_Tag', 'Reviewer_Score']]
    
    #return the data
    return review_data_final

[nltk_data] Downloading package punkt to /Users/coreyreid/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/coreyreid/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/coreyreid/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/coreyreid/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/coreyreid/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [6]:
# Now, I will create a similar function without POS Tagging and we will compare two techniques
# with and without POS Tagging

def preprocess_no_pos(review):
    
    # remove special characters and ensure it is lowercase
    def preprocess_remove_special(review):
        review_handled = re.sub(r'[^a-zA-Z\s]', '', review)
        review_lower = review_handled.lower()
        return review_lower

    #tokenize
    def preprocess_tokenize(review):
        bag_of_words = word_tokenize(review)
        return bag_of_words

    #remove stopwords
    def preprocess_stopwords(review):
        words = stopwords.words('english')
        set_stop_words = set(words)
        removed_stopwords_words = []
        for word in review:
            if word not in set_stop_words:
                removed_stopwords_words.append(word)
        return removed_stopwords_words

    #lemmatize
    def preprocess_lemmatize(review):
        lemmatizer = WordNetLemmatizer()
        lemmatized_review = [lemmatizer.lemmatize(word) for word in review ]
        return lemmatized_review

    #execute functions within the bigger function
    review_data_copy = review_data.copy()
    review_data_copy['Total_Review_handled'] = review_data_copy["Total_Review"].apply(preprocess_remove_special)
    review_data_copy['Total_Review_handled'] = review_data_copy['Total_Review_handled'].apply(preprocess_tokenize)
    review_data_copy['Total_Review_handled'] = review_data_copy['Total_Review_handled'].apply(preprocess_stopwords)
    review_data_copy['Total_Review_handled'] = review_data_copy['Total_Review_handled'].apply(preprocess_lemmatize)

    #finalize data
    review_data_final = review_data_copy[['Total_Review_handled', 'Reviewer_Score']]

    #return the preprocessed data
    return review_data_final

In [7]:
# #next I will process the data with POS Tagging to compare to the previous section
# #these should be the same as it is just an optimized function
preprocessed_reviews_pos_tagging = preprocess_pos(review_data)
preprocessed_reviews_pos_tagging

,POS_Tag,Reviewer_Score
0,"[(angry, JJ), (made, VBD), (post, NN), (availa...",2.9
1,"[(negative, JJ), (real, JJ), (complaint, NN), ...",7.5
2,"[(room, NN), (nice, RB), (elderly, JJ), (bit, ...",7.1
3,"[(room, NN), (dirty, NN), (afraid, JJ), (walk,...",3.8
4,"[(booked, VBN), (company, NN), (line, NN), (sh...",6.7
...,...,...
515733,"[(trolly, RB), (staff, NN), (help, NN), (take,...",7.0
515734,"[(hotel, NN), (look, NN), (like, IN), (surely,...",5.8
515735,"[(ac, JJ), (useless, JJ), (hot, JJ), (week, NN...",2.5
515736,"[(negative, JJ), (room, NN), (enormous, JJ), (...",8.8


In [8]:
#last, I will process the review data without POS Tagging, the processing will end at lemmatizing. 
preprocessed_reviews_no_pos_tagging = preprocess_no_pos(review_data)
preprocessed_reviews_no_pos_tagging

,Total_Review_handled,Reviewer_Score
0,"[angry, made, post, available, via, possible, ...",2.9
1,"[negative, real, complaint, hotel, great, grea...",7.5
2,"[room, nice, elderly, bit, difficult, room, tw...",7.1
3,"[room, dirty, afraid, walk, barefoot, floor, l...",3.8
4,"[booked, company, line, showed, picture, room,...",6.7
...,...,...
515733,"[trolly, staff, help, take, luggage, room, loc...",7.0
515734,"[hotel, look, like, surely, breakfast, ok, got...",5.8
515735,"[ac, useless, hot, week, vienna, gave, hot, ai...",2.5
515736,"[negative, room, enormous, really, comfortable...",8.8


These two dataframes will be the two types of preprocessing methods I will consider in each of the models. This will show the difference between using POS tags and not using them. Additionally, we will now have two cases for exploring the impact on machine learning outcomes. 

The dataframes both are showing the data as expected so we can now proceed to implementing different machine learning models. We will then compare the results and determine which machine learning model is best for sentiment analysis of the Short-Term Rental review data. 

## Part 4: Classifying the Review Score Data for Machine Learning Labels

In [9]:
# first I will define the thresholds - our labels will be positive, negative, and neutral 
# I decided to use a small window for neutral, between 4.5 and 5.5 - this will ensure most the review classifications
# are sensative - it can be updated later if desired by changing the following threshold values

positive_threshold = 5.5
negative_threshold = 4.5

# next, I will classify by building a classification function

def classify_scores(score_value):
    if score_value >= positive_threshold:
        return 'positive'
    elif score_value <= negative_threshold:
        return 'negative'
    else:
        return 'neutral'
    
preprocessed_reviews_pos_tagging_final = preprocessed_reviews_pos_tagging.copy()
    
preprocessed_reviews_pos_tagging_final['Reviewer_Score'] = preprocessed_reviews_pos_tagging_final['Reviewer_Score'].apply(classify_scores)

preprocessed_reviews_pos_tagging_final

,POS_Tag,Reviewer_Score
0,"[(angry, JJ), (made, VBD), (post, NN), (availa...",negative
1,"[(negative, JJ), (real, JJ), (complaint, NN), ...",positive
2,"[(room, NN), (nice, RB), (elderly, JJ), (bit, ...",positive
3,"[(room, NN), (dirty, NN), (afraid, JJ), (walk,...",negative
4,"[(booked, VBN), (company, NN), (line, NN), (sh...",positive
...,...,...
515733,"[(trolly, RB), (staff, NN), (help, NN), (take,...",positive
515734,"[(hotel, NN), (look, NN), (like, IN), (surely,...",positive
515735,"[(ac, JJ), (useless, JJ), (hot, JJ), (week, NN...",negative
515736,"[(negative, JJ), (room, NN), (enormous, JJ), (...",positive


In [10]:
# I will also do the same for the no POS tagging scenario

preprocessed_reviews_no_pos_tagging_final = preprocessed_reviews_no_pos_tagging.copy()
    
preprocessed_reviews_no_pos_tagging_final['Reviewer_Score'] = preprocessed_reviews_no_pos_tagging_final['Reviewer_Score'].apply(classify_scores)

preprocessed_reviews_no_pos_tagging_final

,Total_Review_handled,Reviewer_Score
0,"[angry, made, post, available, via, possible, ...",negative
1,"[negative, real, complaint, hotel, great, grea...",positive
2,"[room, nice, elderly, bit, difficult, room, tw...",positive
3,"[room, dirty, afraid, walk, barefoot, floor, l...",negative
4,"[booked, company, line, showed, picture, room,...",positive
...,...,...
515733,"[trolly, staff, help, take, luggage, room, loc...",positive
515734,"[hotel, look, like, surely, breakfast, ok, got...",positive
515735,"[ac, useless, hot, week, vienna, gave, hot, ai...",negative
515736,"[negative, room, enormous, really, comfortable...",positive


In [11]:
# determine the counts of each category
review_counts = preprocessed_reviews_no_pos_tagging_final['Reviewer_Score'].value_counts()
print(review_counts)

positive    475509
neutral      24188
negative     16041
Name: Reviewer_Score, dtype: int64


The data has now been preprocessed and categorized. As can be seen, the dataset is very unbalanced as most of the data is positive reviews at this threshold. This may be problematic in some of our machine learning models that we will be exploring. There are techniques for unbalanced datasets, so we may need to employ some of those to get better classification. 

## Part 7: Exploring Recurrent Neural Networks (RNN) Outcomes for Sentiment Analysis

### Part 7.1: Libraries and Installations

In this section we will expand to more complex deep learning models to try and get a better performing machine learning model for our sentiment analysis tool. 

In [12]:
# confirm the dataframes are still in order
# preprocessed_reviews_pos_tagging_final
# preprocessed_reviews_no_pos_tagging_final
reviews_pos_tagging_RNN = preprocessed_reviews_pos_tagging_final.copy()
reviews_no_pos_tagging_RNN = preprocessed_reviews_no_pos_tagging_final.copy()

In [13]:
# for this type of model I will use TensorFlow and Keras 
#install the deep learning framework Tensor Flow and library Keras [14][15]
!pip install tensorflow
!pip install keras

In [14]:
#In order to use a Recurrent Neural Network (RNN), we need to do a bit more processing on the data to format it the way it is needed for input
# This includes, seperating the words and the POS tags, adding padding to get the length consistent, and
# creating embeddings for the data, then splitting into test and train datasets for the model. 
#first I will import all the libraries needed for this model - I will use different tokenizers than previously
#to show capabilities with different technologies
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, GRU, Dense, SimpleRNN, Dropout #[16][17]
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

### Part 7.2: Prepare Data for RNN usage

First, we will prepare the POS tagged data to be used in further development tasks below. 

In [15]:
#with an RNN the scores can not be text - they need to be numbers - will encode these now
class LabelEncoderSentiment:
    def __init__(self):
        self.classes_ = ['negative', 'neutral', 'positive']

    def fit_transform(self, labels):
        return np.array([self.classes_.index(label) for label in labels])
    
    def inverse_transform_sentiment(label_encorder, encoded_labels):
        classes_ = ['negative', 'neutral', 'positive']
        return [classes_[label] for label in encoded_labels]

#with an RNN the scores can not be text - they need to be numbers - will encode these now
label_encoder_RNN = LabelEncoderSentiment()
encoded_scores_RNN = label_encoder_RNN.fit_transform(reviews_pos_tagging_RNN["Reviewer_Score"])

# get the text and POS data
review_RNN = reviews_pos_tagging_RNN["POS_Tag"]

# prepare the text and POS data for RNN usage
tokenizer_RNN = Tokenizer()
tokenizer_RNN.fit_on_texts([" ".join([rev for rev, _ in seq]) for seq in review_RNN])
sequences_RNN = tokenizer_RNN.texts_to_sequences([" ".join([rev for rev, _ in seq]) for seq in review_RNN])
review_prepped_RNN = pad_sequences(sequences_RNN, maxlen=10)

# prepare the test and train datasets
X_train_RNN, X_test_RNN, y_train_RNN, y_test_RNN = train_test_split(review_prepped_RNN, encoded_scores_RNN, test_size=0.2, random_state=42)

Next, we will prepare the baseline data for the no POS tag scenario. 

In [16]:
#with an RNN the scores can not be text - they need to be numbers - will encode these now
class LabelEncoderSentiment_noPOS:
    def __init__(self):
        self.classes_ = ['negative', 'neutral', 'positive']

    def fit_transform_noPOS(self, labels):
        return np.array([self.classes_.index(label) for label in labels])
    
    def inverse_transform_sentiment_noPOS(label_encorder, encoded_labels):
        classes_ = ['negative', 'neutral', 'positive']
        return [classes_[label] for label in encoded_labels]

label_encoder_RNN_noPOS = LabelEncoderSentiment_noPOS()
encoded_scores_RNN_noPOS = label_encoder_RNN_noPOS.fit_transform_noPOS(reviews_no_pos_tagging_RNN["Reviewer_Score"])

# get the text and POS data
review_RNN_noPOS = reviews_no_pos_tagging_RNN["Total_Review_handled"]

# prepare the text and POS data for RNN usage
tokenizer_RNN_noPOS = Tokenizer()
tokenizer_RNN_noPOS.fit_on_texts([" ".join(seq) for seq in review_RNN_noPOS])
sequences_RNN_noPOS = tokenizer_RNN_noPOS.texts_to_sequences([" ".join(seq) for seq in review_RNN_noPOS])
review_prepped_RNN_noPOS = pad_sequences(sequences_RNN_noPOS, maxlen=10)

# prepare the test and train datasets
X_train_RNN_noPOS, X_test_RNN_noPOS, y_train_RNN_noPOS, y_test_RNN_noPOS = train_test_split(review_prepped_RNN_noPOS, encoded_scores_RNN_noPOS, test_size=0.2, random_state=42)

### Part 7.3: Build the Baseline Model

POS Tagged Data and Simple Model for Baseline evaluation. 

In [17]:
# # first, I will create a baseline model for the RNN test scenario with POS tags included.
# # it will just be a simple model to begin with
# model_simpleRNN = Sequential()
# model_simpleRNN.add(Embedding(input_dim=len(tokenizer_RNN.word_index) + 1, output_dim=128, input_length=10))
# model_simpleRNN.add(SimpleRNN(units=50))
# model_simpleRNN.add(Dense(units=3, activation='softmax'))

# #compile the model
# model_simpleRNN.compile(
#     loss='sparse_categorical_crossentropy', 
#     optimizer='adam', 
#     metrics=['accuracy'],    
#     weighted_metrics=['accuracy']
# )

# #fit the model
# model_simpleRNN.fit(X_train_RNN, y_train_RNN, epochs=10, batch_size=64, validation_split=0.2)

# pred_simpleRNN = model_simpleRNN.predict(X_test_RNN)

Epoch 1/10
5158/5158 [==============================] - 78s 15ms/step - loss: 0.2468 - accuracy: 0.9258 - weighted_accuracy: 0.9258 - val_loss: 0.2369 - val_accuracy: 0.9255 - val_weighted_accuracy: 0.9255
Epoch 2/10
5158/5158 [==============================] - 80s 16ms/step - loss: 0.2147 - accuracy: 0.9301 - weighted_accuracy: 0.9301 - val_loss: 0.2452 - val_accuracy: 0.9251 - val_weighted_accuracy: 0.9251
Epoch 3/10
5158/5158 [==============================] - 80s 16ms/step - loss: 0.1848 - accuracy: 0.9390 - weighted_accuracy: 0.9390 - val_loss: 0.2564 - val_accuracy: 0.9201 - val_weighted_accuracy: 0.9201
Epoch 4/10
5158/5158 [==============================] - 80s 16ms/step - loss: 0.1579 - accuracy: 0.9479 - weighted_accuracy: 0.9479 - val_loss: 0.2759 - val_accuracy: 0.9189 - val_weighted_accuracy: 0.9189
Epoch 5/10
5158/5158 [==============================] - 80s 16ms/step - loss: 0.1364 - accuracy: 0.9554 - weighted_accuracy: 0.9554 - val_loss: 0.3047 - val_accuracy: 0.9130 - 

In [18]:
# #classification metrics didnt work out as we are getting probabilities of each classification from the model
# #to fix this, we will convert it to the predicted outcome using numpy [19]
# pred_simpleRNN_converted = np.argmax(pred_simpleRNN, axis=1)

# #evaluate results from the RNN
# baseline_simpleRNN_report_POS = classification_report(y_test_RNN, pred_simpleRNN_converted)
# baseline_simpleRNN_confusion_matrix_POS = confusion_matrix(y_test_RNN, pred_simpleRNN_converted)
# baseline_simpleRNN_accuracy_POS = accuracy_score(y_test_RNN, pred_simpleRNN_converted)

# #print the results
# print("Classification Report:\n", baseline_simpleRNN_report_POS)
# print("Confusion Matrix:\n", baseline_simpleRNN_confusion_matrix_POS)
# print("Accuracy:", baseline_simpleRNN_accuracy_POS)

Classification Report:
               precision    recall  f1-score   support

           0       0.37      0.27      0.31      3198
           1       0.22      0.14      0.17      4981
           2       0.94      0.97      0.96     94969

    accuracy                           0.91    103148
   macro avg       0.51      0.46      0.48    103148
weighted avg       0.89      0.91      0.90    103148

Confusion Matrix:
 [[  865   545  1788]
 [  482   685  3814]
 [  994  1899 92076]]
Accuracy: 0.907686043355179


No POS Tags and Model Developed for the baseline in this scenario

In [19]:
# # next, I will create a baseline model for the RNN test scenario with no POS tags included.
# # it will just be a simple model to begin with
# model_simpleRNN_noPOS = Sequential()
# model_simpleRNN_noPOS.add(Embedding(input_dim=len(tokenizer_RNN_noPOS.word_index) + 1, output_dim=128, input_length=10))
# model_simpleRNN_noPOS.add(SimpleRNN(units=50))
# model_simpleRNN_noPOS.add(Dense(units=3, activation='softmax'))

# #compile the model
# model_simpleRNN_noPOS.compile(
#     loss='sparse_categorical_crossentropy', 
#     optimizer='adam', 
#     metrics=['accuracy'],    
#     weighted_metrics=['accuracy']
# )

# #fit the model
# model_simpleRNN_noPOS.fit(X_train_RNN_noPOS, y_train_RNN_noPOS, epochs=10, batch_size=64, validation_split=0.2)

# pred_simpleRNN_noPOS = model_simpleRNN_noPOS.predict(X_test_RNN_noPOS)

Epoch 1/10
5158/5158 [==============================] - 80s 15ms/step - loss: 0.2488 - accuracy: 0.9254 - weighted_accuracy: 0.9254 - val_loss: 0.2408 - val_accuracy: 0.9239 - val_weighted_accuracy: 0.9239
Epoch 2/10
5158/5158 [==============================] - 80s 16ms/step - loss: 0.2137 - accuracy: 0.9307 - weighted_accuracy: 0.9307 - val_loss: 0.2430 - val_accuracy: 0.9237 - val_weighted_accuracy: 0.9237
Epoch 3/10
5158/5158 [==============================] - 81s 16ms/step - loss: 0.1819 - accuracy: 0.9400 - weighted_accuracy: 0.9400 - val_loss: 0.2572 - val_accuracy: 0.9218 - val_weighted_accuracy: 0.9218
Epoch 4/10
5158/5158 [==============================] - 81s 16ms/step - loss: 0.1540 - accuracy: 0.9497 - weighted_accuracy: 0.9497 - val_loss: 0.2822 - val_accuracy: 0.9141 - val_weighted_accuracy: 0.9141
Epoch 5/10
5158/5158 [==============================] - 81s 16ms/step - loss: 0.1320 - accuracy: 0.9569 - weighted_accuracy: 0.9569 - val_loss: 0.3025 - val_accuracy: 0.9120 - 

In [20]:
# #classification metrics didnt work out as we are getting probabilities of each classification from the model
# #to fix this, we will convert it to the predicted outcome using numpy [19]
# pred_simpleRNN_noPOS_converted = np.argmax(pred_simpleRNN_noPOS, axis=1)

# #evaluate results from the RNN
# baseline_simpleRNN_noPOS_report_POS = classification_report(y_test_RNN_noPOS, pred_simpleRNN_noPOS_converted)
# baseline_simpleRNN_noPOS_confusion_matrix_POS = confusion_matrix(y_test_RNN_noPOS, pred_simpleRNN_noPOS_converted)
# baseline_simpleRNN_noPOS_accuracy_POS = accuracy_score(y_test_RNN_noPOS, pred_simpleRNN_noPOS_converted)

# #print the results
# print("Classification Report:\n", baseline_simpleRNN_noPOS_report_POS)
# print("Confusion Matrix:\n", baseline_simpleRNN_noPOS_confusion_matrix_POS)
# print("Accuracy:", baseline_simpleRNN_noPOS_accuracy_POS)

Classification Report:
               precision    recall  f1-score   support

           0       0.35      0.26      0.30      3198
           1       0.20      0.11      0.14      4981
           2       0.94      0.97      0.96     94969

    accuracy                           0.91    103148
   macro avg       0.50      0.45      0.46    103148
weighted avg       0.89      0.91      0.90    103148

Confusion Matrix:
 [[  832   442  1924]
 [  477   530  3974]
 [ 1039  1704 92226]]
Accuracy: 0.9073176406716562


This concludes the baseline model approximations and we can see that in both cases, the results seem to be overfitting the data. This can be seen because it is only getting reasonable accuracy, precision, and recall values for the positive case. This is likely due to the imbalanced dataset that we are working with. 

### 7.4: Testing Imbalanced Data Fixes to Determine Best Approach

#### 7.4.1: Using SMOTE to resample the data and try and improve the imbalance

In [17]:
# in order to try and improve the deep learning prediction model, I am going to implement 
# synthetic minority over-sampling technique (SMOTE) to try and produce more minority class data. 
#SMOTE generates samples fo rus to help balance the dataset. 

#first I will load my libraries in
!pip install -U imbalanced-learn
from imblearn.over_sampling import SMOTE

Starting with the POS Tagged Data

In [22]:
# #I will oversample the data as the minority class is not being recognized well
# # this was shown by training the model without any additional fine tuning and just the baseline preprocessing. 
# smote_RNN = SMOTE(sampling_strategy='auto', random_state=42)
# X_train_RNN_resampled, y_train_RNN_resampled = smote_RNN.fit_resample(X_train_RNN, y_train_RNN)

# model_simpleRNN_smote = Sequential()
# model_simpleRNN_smote.add(Embedding(input_dim=len(tokenizer_RNN.word_index) + 1, output_dim=128, input_length=10))
# model_simpleRNN_smote.add(SimpleRNN(units=50))
# model_simpleRNN_smote.add(Dense(units=3, activation='softmax'))

# #compile the model
# model_simpleRNN_smote.compile(
#     loss='sparse_categorical_crossentropy', 
#     optimizer='adam', 
#     metrics=['accuracy'],    
#     weighted_metrics=['accuracy']
# )

# #fit the model
# model_simpleRNN_smote.fit(X_train_RNN_resampled, y_train_RNN_resampled, epochs=10, batch_size=64, validation_split=0.2)

# pred_simpleRNN_smote = model_simpleRNN_smote.predict(X_test_RNN)

Epoch 1/10
14271/14271 [==============================] - 223s 16ms/step - loss: 0.6367 - accuracy: 0.7334 - weighted_accuracy: 0.7334 - val_loss: 1.4128 - val_accuracy: 0.0103 - val_weighted_accuracy: 0.0103
Epoch 2/10
14271/14271 [==============================] - 227s 16ms/step - loss: 0.5328 - accuracy: 0.7764 - weighted_accuracy: 0.7764 - val_loss: 1.3718 - val_accuracy: 0.0329 - val_weighted_accuracy: 0.0329
Epoch 3/10
14271/14271 [==============================] - 228s 16ms/step - loss: 0.4959 - accuracy: 0.7890 - weighted_accuracy: 0.7890 - val_loss: 1.3003 - val_accuracy: 0.0686 - val_weighted_accuracy: 0.0686
Epoch 4/10
14271/14271 [==============================] - 230s 16ms/step - loss: 0.4699 - accuracy: 0.7987 - weighted_accuracy: 0.7987 - val_loss: 1.4344 - val_accuracy: 0.0484 - val_weighted_accuracy: 0.0484
Epoch 5/10
14271/14271 [==============================] - 228s 16ms/step - loss: 0.4503 - accuracy: 0.8064 - weighted_accuracy: 0.8064 - val_loss: 1.4498 - val_accu

In [23]:
# pred_simpleRNN_converted_resampled = np.argmax(pred_simpleRNN_smote, axis=1)

# #evaluate results from the RNN
# baseline_simpleRNN_report_POS = classification_report(y_test_RNN, pred_simpleRNN_converted_resampled)
# baseline_simpleRNN_confusion_matrix_POS = confusion_matrix(y_test_RNN, pred_simpleRNN_converted_resampled)
# baseline_simpleRNN_accuracy_POS = accuracy_score(y_test_RNN, pred_simpleRNN_converted_resampled)

# #print the results
# print("Classification Report:\n", baseline_simpleRNN_report_POS)
# print("Confusion Matrix:\n", baseline_simpleRNN_confusion_matrix_POS)
# print("Accuracy:", baseline_simpleRNN_accuracy_POS)

Classification Report:
               precision    recall  f1-score   support

           0       0.15      0.42      0.23      3198
           1       0.14      0.10      0.12      4981
           2       0.95      0.90      0.92     94969

    accuracy                           0.85    103148
   macro avg       0.41      0.48      0.42    103148
weighted avg       0.88      0.85      0.86    103148

Confusion Matrix:
 [[ 1349   347  1502]
 [ 1093   509  3379]
 [ 6317  2794 85858]]
Accuracy: 0.8503897312599372


Next, I will perform the same but for the no POS tagging case

In [24]:
# #I will oversample the data as the minority class is not being recognized well
# # this was shown by training the model without any additional fine tuning and just the baseline preprocessing. 

# smote_RNN_noPOS = SMOTE(sampling_strategy='not majority', random_state=42)
# X_train_RNN_noPOS_resampled, y_train_RNN_noPOS_resampled = smote_RNN_noPOS.fit_resample(X_train_RNN_noPOS, y_train_RNN_noPOS)

# model_simpleRNN_smote_noPOS = Sequential()
# model_simpleRNN_smote_noPOS.add(Embedding(input_dim=len(tokenizer_RNN_noPOS.word_index) + 1, output_dim=128, input_length=10))
# model_simpleRNN_smote_noPOS.add(SimpleRNN(units=50))
# model_simpleRNN_smote_noPOS.add(Dense(units=3, activation='softmax'))


# #compile the model
# model_simpleRNN_smote_noPOS.compile(
#     loss='sparse_categorical_crossentropy', 
#     optimizer='adam', 
#     metrics=['accuracy'],    
#     weighted_metrics=['accuracy']
# )

# #fit the model
# model_simpleRNN_smote_noPOS.fit(X_train_RNN_noPOS_resampled, y_train_RNN_noPOS_resampled, epochs=10, batch_size=64, validation_split=0.2)

# pred_simpleRNN_smote_noPOS = model_simpleRNN_smote_noPOS.predict(X_test_RNN_noPOS)

Epoch 1/10
14271/14271 [==============================] - 227s 16ms/step - loss: 0.6395 - accuracy: 0.7323 - weighted_accuracy: 0.7323 - val_loss: 1.3930 - val_accuracy: 0.0159 - val_weighted_accuracy: 0.0159
Epoch 2/10
14271/14271 [==============================] - 228s 16ms/step - loss: 0.5317 - accuracy: 0.7771 - weighted_accuracy: 0.7771 - val_loss: 1.3863 - val_accuracy: 0.0284 - val_weighted_accuracy: 0.0284
Epoch 3/10
14271/14271 [==============================] - 241s 17ms/step - loss: 0.4944 - accuracy: 0.7899 - weighted_accuracy: 0.7899 - val_loss: 1.3857 - val_accuracy: 0.0417 - val_weighted_accuracy: 0.0417
Epoch 4/10
14271/14271 [==============================] - 242s 17ms/step - loss: 0.4689 - accuracy: 0.7990 - weighted_accuracy: 0.7990 - val_loss: 1.3570 - val_accuracy: 0.0954 - val_weighted_accuracy: 0.0954
Epoch 5/10
14271/14271 [==============================] - 241s 17ms/step - loss: 0.4497 - accuracy: 0.8062 - weighted_accuracy: 0.8062 - val_loss: 1.4197 - val_accu

In [25]:
# pred_simpleRNN_noPOS_converted_resampled = np.argmax(pred_simpleRNN_smote_noPOS, axis=1)

# #evaluate results from the RNN
# baseline_simpleRNN_report_noPOS = classification_report(y_test_RNN_noPOS, pred_simpleRNN_noPOS_converted_resampled)
# baseline_simpleRNN_confusion_matrix_noPOS = confusion_matrix(y_test_RNN_noPOS, pred_simpleRNN_noPOS_converted_resampled)
# baseline_simpleRNN_accuracy_noPOS = accuracy_score(y_test_RNN_noPOS, pred_simpleRNN_noPOS_converted_resampled)

# #print the results
# print("Classification Report:\n", baseline_simpleRNN_report_noPOS)
# print("Confusion Matrix:\n", baseline_simpleRNN_confusion_matrix_noPOS)
# print("Accuracy:", baseline_simpleRNN_accuracy_noPOS)

Classification Report:
               precision    recall  f1-score   support

           0       0.15      0.45      0.23      3198
           1       0.15      0.14      0.14      4981
           2       0.95      0.89      0.92     94969

    accuracy                           0.84    103148
   macro avg       0.42      0.49      0.43    103148
weighted avg       0.89      0.84      0.86    103148

Confusion Matrix:
 [[ 1434   437  1327]
 [ 1174   675  3132]
 [ 6701  3302 84966]]
Accuracy: 0.8441753596773568


In this case, the noPOS still is performing better, but the data still seems very overfit (this was confirmed by testing on the review types below as well). The data seems to now be overfit to the negative class though, so SMOTE has introduced some noise. Moving forward, while iterating through different techniques, I will only use the noPOS scenario, since throughout the report so far it has shown that the difference is very small, and the noPOS usually performs better. This will allow me to iterate through new methods more efficiently. 

#### 7.4.2: Adding L2 Normalization to the SMOTE resampled data to try and lower the potential for overfitting

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.regularizers import l2

In [26]:
# model_simpleRNN_Normalized = Sequential()
# model_simpleRNN_Normalized.add(Embedding(input_dim=len(tokenizer_RNN_noPOS.word_index) + 1, output_dim=128, input_length=10))
# model_simpleRNN_Normalized.add(SimpleRNN(units=50))
# model_simpleRNN_Normalized.add(Dense(64, activation='relu', kernel_regularizer=l2(0.0001), input_dim=(10,1)))
# model_simpleRNN_Normalized.add(Dense(32, activation='relu', kernel_regularizer=l2(0.0001)))
# model_simpleRNN_Normalized.add(Dense(3, activation='softmax'))

# model_simpleRNN_Normalized.compile(
#     loss='sparse_categorical_crossentropy', 
#     optimizer='adam', 
#     metrics=['accuracy'],    
#     weighted_metrics=['accuracy']
# )

# model_simpleRNN_Normalized.fit(X_train_RNN_noPOS_resampled, y_train_RNN_noPOS_resampled, epochs=10, batch_size=64, validation_split=0.2)

Epoch 1/10
14271/14271 [==============================] - 244s 17ms/step - loss: 0.6392 - accuracy: 0.7350 - weighted_accuracy: 0.7350 - val_loss: 1.4643 - val_accuracy: 0.0118 - val_weighted_accuracy: 0.0118
Epoch 2/10
14271/14271 [==============================] - 241s 17ms/step - loss: 0.5381 - accuracy: 0.7765 - weighted_accuracy: 0.7765 - val_loss: 1.4285 - val_accuracy: 0.0134 - val_weighted_accuracy: 0.0134
Epoch 3/10
14271/14271 [==============================] - 245s 17ms/step - loss: 0.5028 - accuracy: 0.7887 - weighted_accuracy: 0.7887 - val_loss: 1.4156 - val_accuracy: 0.0113 - val_weighted_accuracy: 0.0113
Epoch 4/10
14271/14271 [==============================] - 244s 17ms/step - loss: 0.4793 - accuracy: 0.7963 - weighted_accuracy: 0.7963 - val_loss: 1.3780 - val_accuracy: 0.0689 - val_weighted_accuracy: 0.0689
Epoch 5/10
14271/14271 [==============================] - 248s 17ms/step - loss: 0.4611 - accuracy: 0.8030 - weighted_accuracy: 0.8030 - val_loss: 1.4821 - val_accu

In [27]:
# # test predictions and get evaluation metrics
# pred_simpleRNN_noPOS_normalized = model_simpleRNN_Normalized.predict(X_test_RNN_noPOS)

# #convert back
# pred_simpleRNN_noPOS_converted_normalized = np.argmax(pred_simpleRNN_noPOS_normalized, axis=1)

# #evaluate results from the RNN
# normalized_simpleRNN_report_noPOS = classification_report(y_test_RNN_noPOS, pred_simpleRNN_noPOS_converted_normalized)
# normalized_simpleRNN_confusion_matrix_noPOS = confusion_matrix(y_test_RNN_noPOS, pred_simpleRNN_noPOS_converted_normalized)
# normalized_simpleRNN_accuracy_noPOS = accuracy_score(y_test_RNN_noPOS, pred_simpleRNN_noPOS_converted_normalized)

# #print the results
# print("Classification Report:\n", normalized_simpleRNN_report_noPOS)
# print("Confusion Matrix:\n", normalized_simpleRNN_confusion_matrix_noPOS)
# print("Accuracy:", normalized_simpleRNN_accuracy_noPOS)

3224/3224 [==============================] - 1s 411us/step
Classification Report:
               precision    recall  f1-score   support

           0       0.16      0.45      0.24      3198
           1       0.16      0.08      0.11      4981
           2       0.95      0.91      0.93     94969

    accuracy                           0.86    103148
   macro avg       0.42      0.48      0.43    103148
weighted avg       0.88      0.86      0.87    103148

Confusion Matrix:
 [[ 1434   268  1496]
 [ 1167   408  3406]
 [ 6184  1945 86840]]
Accuracy: 0.8597549152673828


#### Part 7.4.3: Undersampling the majority class and using L2 Normalization

In [19]:
#import RandomUnderSampler
from imblearn.under_sampling import RandomUnderSampler

In [28]:
# under_sampler = RandomUnderSampler(sampling_strategy='majority', random_state=42)
# X_train_undersampled, y_train_undersampled = under_sampler.fit_resample(X_train_RNN_noPOS, y_train_RNN_noPOS)

# model_simpleRNN_normalized_undersampled = Sequential()
# model_simpleRNN_normalized_undersampled.add(Embedding(input_dim=len(tokenizer_RNN_noPOS.word_index) + 1, output_dim=128, input_length=10))
# model_simpleRNN_normalized_undersampled.add(SimpleRNN(units=50))
# model_simpleRNN_normalized_undersampled.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), input_dim=(10,1)))
# model_simpleRNN_normalized_undersampled.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
# model_simpleRNN_normalized_undersampled.add(Dense(3, activation='softmax'))

# model_simpleRNN_normalized_undersampled.compile(
#     loss='sparse_categorical_crossentropy', 
#     optimizer='adam', 
#     metrics=['accuracy'],    
#     weighted_metrics=['accuracy']
# )

# model_simpleRNN_normalized_undersampled.fit(X_train_undersampled, y_train_undersampled, epochs=10, batch_size=64, validation_split=0.2)

# # test predictions and get evaluation metrics
# pred_simpleRNN_noPOS_undersampled = model_simpleRNN_normalized_undersampled.predict(X_test_RNN_noPOS)

# #convert back
# pred_simpleRNN_noPOS_converted_undersampled = np.argmax(pred_simpleRNN_noPOS_undersampled, axis=1)

# #evaluate results from the RNN
# undersampled_normalized_simpleRNN_report_noPOS = classification_report(y_test_RNN_noPOS, pred_simpleRNN_noPOS_converted_undersampled)
# undersampled_normalized_simpleRNN_confusion_matrix_noPOS = confusion_matrix(y_test_RNN_noPOS, pred_simpleRNN_noPOS_converted_undersampled)
# undersampled_normalized_simpleRNN_accuracy_noPOS = accuracy_score(y_test_RNN_noPOS, pred_simpleRNN_noPOS_converted_undersampled)

# #print the results
# print("Classification Report:\n", undersampled_normalized_simpleRNN_report_noPOS)
# print("Confusion Matrix:\n", undersampled_normalized_simpleRNN_confusion_matrix_noPOS)
# print("Accuracy:", undersampled_normalized_simpleRNN_accuracy_noPOS)

Epoch 1/10
562/562 [==============================] - 9s 16ms/step - loss: 1.0122 - accuracy: 0.5929 - weighted_accuracy: 0.5929 - val_loss: 1.4901 - val_accuracy: 0.4402 - val_weighted_accuracy: 0.4402
Epoch 2/10
562/562 [==============================] - 9s 16ms/step - loss: 0.7480 - accuracy: 0.6722 - weighted_accuracy: 0.6722 - val_loss: 1.6932 - val_accuracy: 0.3386 - val_weighted_accuracy: 0.3386
Epoch 3/10
562/562 [==============================] - 9s 16ms/step - loss: 0.6158 - accuracy: 0.7540 - weighted_accuracy: 0.7540 - val_loss: 1.5607 - val_accuracy: 0.4400 - val_weighted_accuracy: 0.4400
Epoch 4/10
562/562 [==============================] - 9s 16ms/step - loss: 0.4736 - accuracy: 0.8271 - weighted_accuracy: 0.8271 - val_loss: 1.7571 - val_accuracy: 0.4343 - val_weighted_accuracy: 0.4343
Epoch 5/10
562/562 [==============================] - 9s 16ms/step - loss: 0.3744 - accuracy: 0.8682 - weighted_accuracy: 0.8682 - val_loss: 2.2881 - val_accuracy: 0.3540 - val_weighted_ac

#### 7.4.4: No over or under sampling, just L2 Normalization

In [37]:
model_simpleRNN_normalized_only = Sequential()
model_simpleRNN_normalized_only.add(Embedding(input_dim=len(tokenizer_RNN_noPOS.word_index) + 1, output_dim=128, input_length=10))
model_simpleRNN_normalized_only.add(SimpleRNN(units=50))
model_simpleRNN_normalized_only.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), input_dim=(10,1)))
model_simpleRNN_normalized_only.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
model_simpleRNN_normalized_only.add(Dense(3, activation='softmax'))

model_simpleRNN_normalized_only.compile(
    loss='sparse_categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy'],    
    weighted_metrics=['accuracy']
)

model_simpleRNN_normalized_only.fit(X_train_RNN_noPOS, y_train_RNN_noPOS, epochs=10, batch_size=64, validation_split=0.2)

# test predictions and get evaluation metrics
pred_simpleRNN_noPOS_normalized_only = model_simpleRNN_normalized_only.predict(X_test_RNN_noPOS)

#convert back
pred_simpleRNN_noPOS_normalized_only = np.argmax(pred_simpleRNN_noPOS_normalized_only, axis=1)

#evaluate results from the RNN
only_normalized_simpleRNN_report_noPOS = classification_report(y_test_RNN_noPOS, pred_simpleRNN_noPOS_normalized_only)
only_normalized_simpleRNN_confusion_matrix_noPOS = confusion_matrix(y_test_RNN_noPOS, pred_simpleRNN_noPOS_normalized_only)
only_normalized_simpleRNN_accuracy_noPOS = accuracy_score(y_test_RNN_noPOS, pred_simpleRNN_noPOS_normalized_only)

#print the results
print("Classification Report:\n", only_normalized_simpleRNN_report_noPOS)
print("Confusion Matrix:\n", only_normalized_simpleRNN_confusion_matrix_noPOS)
print("Accuracy:", only_normalized_simpleRNN_accuracy_noPOS)

Epoch 1/10
5158/5158 [==============================] - 80s 15ms/step - loss: 0.2804 - accuracy: 0.9219 - weighted_accuracy: 0.9219 - val_loss: 0.2528 - val_accuracy: 0.9219 - val_weighted_accuracy: 0.9219
Epoch 2/10
5158/5158 [==============================] - 82s 16ms/step - loss: 0.2305 - accuracy: 0.9276 - weighted_accuracy: 0.9276 - val_loss: 0.2450 - val_accuracy: 0.9244 - val_weighted_accuracy: 0.9244
Epoch 3/10
5158/5158 [==============================] - 81s 16ms/step - loss: 0.2118 - accuracy: 0.9308 - weighted_accuracy: 0.9308 - val_loss: 0.2513 - val_accuracy: 0.9252 - val_weighted_accuracy: 0.9252
Epoch 4/10
5158/5158 [==============================] - 81s 16ms/step - loss: 0.1953 - accuracy: 0.9338 - weighted_accuracy: 0.9338 - val_loss: 0.2565 - val_accuracy: 0.9223 - val_weighted_accuracy: 0.9223
Epoch 5/10
5158/5158 [==============================] - 81s 16ms/step - loss: 0.1810 - accuracy: 0.9378 - weighted_accuracy: 0.9378 - val_loss: 0.2638 - val_accuracy: 0.9172 - 

#### 7.4.5: Multiclass SMOTE oversampling for both negative and neutral and LSTM Model with L2 regularization

In [30]:
# #get the totals for multiclass smote
# unique_values, counts = np.unique(y_train_RNN_noPOS, return_counts=True)
# #print the values to check 
# for label, count in zip(unique_values, counts):
#     print(f"Class '{label}': {count} occurrences")

# #update the sampling dict for the smote analysis
# sampling_dict = {0:190270, 1:190270, 2:380540}
# smote_multi_RNN_noPOS = SMOTE(sampling_strategy=sampling_dict, random_state=42)
# X_train_RNN_noPOS_multi, y_train_RNN_noPOS_multi = smote_multi_RNN_noPOS.fit_resample(X_train_RNN_noPOS, y_train_RNN_noPOS)

# unique_values_multi, counts_multi = np.unique(y_train_RNN_noPOS_multi, return_counts=True)

# for label, count in zip(unique_values_multi, counts_multi):
#     print(f"Class '{label}': {count} occurrences")

# model_simpleRNN_smote_multi_noPOS = Sequential()
# model_simpleRNN_smote_multi_noPOS.add(Embedding(input_dim=len(tokenizer_RNN_noPOS.word_index) + 1, output_dim=128, input_length=10))
# model_simpleRNN_smote_multi_noPOS.add(LSTM(units=50, kernel_regularizer=l2(0.001)))
# model_simpleRNN_smote_multi_noPOS.add(Dense(units=3, activation='softmax'))


# #compile the model
# model_simpleRNN_smote_multi_noPOS.compile(
#     loss='sparse_categorical_crossentropy', 
#     optimizer='adam', 
#     metrics=['accuracy'],    
#     weighted_metrics=['accuracy']
# )

# #fit the model
# model_simpleRNN_smote_multi_noPOS.fit(X_train_RNN_noPOS_multi, y_train_RNN_noPOS_multi, epochs=10, batch_size=64, validation_split=0.2)

# pred_simpleRNN_smote_multi_noPOS = model_simpleRNN_smote_multi_noPOS.predict(X_test_RNN_noPOS)

# pred_simpleRNN_noPOS_multi_resampled = np.argmax(pred_simpleRNN_smote_multi_noPOS, axis=1)

# #evaluate results from the RNN
# multi_simpleRNN_report_noPOS = classification_report(y_test_RNN_noPOS, pred_simpleRNN_noPOS_multi_resampled)
# multi_simpleRNN_confusion_matrix_noPOS = confusion_matrix(y_test_RNN_noPOS, pred_simpleRNN_noPOS_multi_resampled)
# multi_simpleRNN_accuracy_noPOS = accuracy_score(y_test_RNN_noPOS, pred_simpleRNN_noPOS_multi_resampled)

# #print the results
# print("Classification Report:\n", multi_simpleRNN_report_noPOS)
# print("Confusion Matrix:\n", multi_simpleRNN_confusion_matrix_noPOS)
# print("Accuracy:", multi_simpleRNN_accuracy_noPOS)

Class '0': 12843 occurrences
Class '1': 19207 occurrences
Class '2': 380540 occurrences
Class '0': 190270 occurrences
Class '1': 190270 occurrences
Class '2': 380540 occurrences
Epoch 1/10
9514/9514 [==============================] - 155s 16ms/step - loss: 0.5139 - accuracy: 0.8093 - weighted_accuracy: 0.8093 - val_loss: 2.3256 - val_accuracy: 1.2482e-04 - val_weighted_accuracy: 1.2482e-04
Epoch 2/10
9514/9514 [==============================] - 155s 16ms/step - loss: 0.4020 - accuracy: 0.8627 - weighted_accuracy: 0.8627 - val_loss: 2.4474 - val_accuracy: 4.4016e-04 - val_weighted_accuracy: 4.4016e-04
Epoch 3/10
9514/9514 [==============================] - 157s 17ms/step - loss: 0.3663 - accuracy: 0.8773 - weighted_accuracy: 0.8773 - val_loss: 2.4229 - val_accuracy: 3.0877e-04 - val_weighted_accuracy: 3.0877e-04
Epoch 4/10
9514/9514 [==============================] - 160s 17ms/step - loss: 0.3458 - accuracy: 0.8849 - weighted_accuracy: 0.8849 - val_loss: 2.4240 - val_accuracy: 8.8690e-0

In [31]:
# # seemingly the data is always overfitting the training for multiple models I have tried 
# # i need to explore if the data being created is any good from SMOTE

# negative_indices = np.where(y_train_RNN_noPOS_multi == 0)[0]

# for i in range(12483, 12943):
#     index = negative_indices[i]
#     reverse_word_index = {v: k for k, v in tokenizer_RNN_noPOS.word_index.items()}
#     decoded_sequence = [reverse_word_index.get(token, '') for token in X_train_RNN_noPOS_multi[index]]
#     print("Decoded Sequence:", ' '.join(decoded_sequence))

Decoded Sequence: customer badly bad experience breakfast great employee restaurant really nice
Decoded Sequence: investigation pleased manner hotel handled situation good location quiet room
Decoded Sequence:         everything location
Decoded Sequence: coffee shop work price per night totally unacceptable good location
Decoded Sequence: money tiny room air conditioning dated decor good central location
Decoded Sequence: work report morning evening cash machine order location helpful staff
Decoded Sequence: wifi attention request made booking fourth stay last restaurant nice
Decoded Sequence: help always proposing candy kid free apple water always available
Decoded Sequence: room resemble shown website rear garden view bedroom wad pleasant
Decoded Sequence: would change mattress doubt exactly two day sleep awful breakfast
Decoded Sequence: smoking neither room noisey sleep wink either night central location
Decoded Sequence:         nothing nothing
Decoded Sequence: neighbourhood nig

Decoded Sequence:      small noisy room near paddington
Decoded Sequence:    negative never truly uncomfortable huge waste money
Decoded Sequence: club hotel spa relaxing day sunday staff really helpful friendly
Decoded Sequence: location paid gbp night would liked pay half best positive
Decoded Sequence: brush tissue paper check guy wanted charge u friendly positive
Decoded Sequence: arrival furthermore room extremely small even open trolley space positive
Decoded Sequence: last moment competition provides free wifi lead market follow positive
Decoded Sequence: previous stay city enjoyed much better comfort lower price positive
Decoded Sequence: provided better service hotel staff location good worth money paid
Decoded Sequence: room complained said nothink else available think worth money pool
Decoded Sequence: area paris fridge really hard move inside tiny space nothing
Decoded Sequence: cheap cheap hotel holiday inn would better cheaper staff loby
Decoded Sequence: nothing friendly

Decoded Sequence:       sweage coming bathroom nothing
Decoded Sequence:  dislike indifference staff attitude towards costumer demand like location
Decoded Sequence: lead passenger resulting charged fee paying using debit card positive
Decoded Sequence: morning stay room ok size location good working near aldgate
Decoded Sequence:   clup room give bed room dirty room location
Decoded Sequence: week booking com check property promote nothing really bad experience
Decoded Sequence: staff never go back nice location every thing modern clean
Decoded Sequence:      rude staff bathroom filthy positive
Decoded Sequence: properly mattress horrible fridge bathroom seat broken miserable elevator positive
Decoded Sequence:       lousy hotel lousy hotel
Decoded Sequence: cheaper case much better experience absolutely staying neither friend positive
Decoded Sequence: family room make sofa bed bedding supplied ask time positive
Decoded Sequence: decent temp tap stiff breakfast good good choice locat

Decoded Sequence: drop time could bread negative negative none open place amazing
Decoded Sequence: immediately ok walkable access work door room enthusiastic even lyon
Decoded Sequence: london upgraded garden bit area lot central phone socket nothing
Decoded Sequence: privacy stay knocking enough pm central traveller spot comfortable nice
Decoded Sequence: nightmare tube staff able many different nothing gave friend well
Decoded Sequence: first open require detail small enough get location bed lovely
Decoded Sequence: stayed tiny look angle brother market large walking location bit
Decoded Sequence: nest unique credit see contacted ever floor enough could positive
Decoded Sequence: cleaning parking choice pudding room work night lovely light breakfast
Decoded Sequence: dark taking gratuitously need needed well late girl walk important
Decoded Sequence: everything lunch charged many travelling service comfortable staff back may
Decoded Sequence: bed served handy got could view perfect 

There isn't anything crazy unusual with these reviews, as compared to the processed reviews from the original data, so seemingly these are ok for negative reviews. I will need to look into different techniques to try and improve the model prediction. I will try adjusting class weights to penalize the majority class. 

#### 7.4.6: Class weights adjusted and bidirectional LSTM tested with SMOTE

In [32]:
# #adjust the class weights
# class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_RNN_noPOS), y=y_train_RNN_noPOS_multi)
# class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
# sample_weights = np.array([class_weights_dict[y] for y in y_train_RNN_noPOS_multi])

# #will try a bi-directional LSTM model 
# model_RNN_bidirect_class_smote = Sequential()
# model_RNN_bidirect_class_smote.add(Embedding(input_dim=len(tokenizer_RNN_noPOS.word_index) + 1, output_dim=128, input_length=10))
# model_RNN_bidirect_class_smote.add(Bidirectional(LSTM(128)))
# model_RNN_bidirect_class_smote.add(Dense(3, activation='softmax'))

# # Compile the model with the updated class weights
# model_RNN_bidirect_class_smote.compile(
#     loss='sparse_categorical_crossentropy',
#     optimizer='adam',
#     metrics=['accuracy'],
#     weighted_metrics=['accuracy']
# )

# #fit the model
# model_RNN_bidirect_class_smote.fit(X_train_RNN_noPOS_multi, y_train_RNN_noPOS_multi, epochs=10, batch_size=128, validation_split=0.2, sample_weight=sample_weights)

# pred_RNN_bidirect_class_smote = model_RNN_bidirect_class_smote.predict(X_test_RNN_noPOS)

# pred_converted_RNN_bidirect_class_smote = np.argmax(pred_RNN_bidirect_class_smote, axis=1)

# #evaluate results from the RNN
# report_model_RNN_bidirect_class_smote = classification_report(y_test_RNN_noPOS, pred_converted_RNN_bidirect_class_smote)
# confusion_matrix_model_RNN_bidirect_class_smote = confusion_matrix(y_test_RNN_noPOS, pred_converted_RNN_bidirect_class_smote)
# accuracy_model_RNN_bidirect_class_smote = accuracy_score(y_test_RNN_noPOS, pred_converted_RNN_bidirect_class_smote)

# #print the results
# print("Classification Report:\n", report_model_RNN_bidirect_class_smote)
# print("Confusion Matrix:\n", confusion_matrix_model_RNN_bidirect_class_smote)
# print("Accuracy:", accuracy_model_RNN_bidirect_class_smote)

Epoch 1/10
4757/4757 [==============================] - 136s 28ms/step - loss: 0.4982 - accuracy: 0.8153 - weighted_accuracy: 0.7931 - val_loss: 3.0324 - val_accuracy: 4.3359e-04 - val_weighted_accuracy: 4.3359e-04
Epoch 2/10
4757/4757 [==============================] - 134s 28ms/step - loss: 0.3810 - accuracy: 0.8753 - weighted_accuracy: 0.8500 - val_loss: 3.2635 - val_accuracy: 0.0023 - val_weighted_accuracy: 0.0023
Epoch 3/10
4757/4757 [==============================] - 140s 29ms/step - loss: 0.3358 - accuracy: 0.8920 - weighted_accuracy: 0.8676 - val_loss: 3.0088 - val_accuracy: 0.0050 - val_weighted_accuracy: 0.0050
Epoch 4/10
4757/4757 [==============================] - 144s 30ms/step - loss: 0.2974 - accuracy: 0.9044 - weighted_accuracy: 0.8818 - val_loss: 3.0358 - val_accuracy: 0.0246 - val_weighted_accuracy: 0.0246
Epoch 5/10
4757/4757 [==============================] - 135s 28ms/step - loss: 0.2591 - accuracy: 0.9161 - weighted_accuracy: 0.8962 - val_loss: 3.6745 - val_accura

#### 7.4.7: Class weights only with LSTM and L2 Norm

In [33]:
# # next I will try a new model architecture with class weights adjusted and L2 norm 
# class_weights_lstm = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_RNN_noPOS), y=y_train_RNN_noPOS)
# class_weights_dict = {i: weight for i, weight in enumerate(class_weights_lstm)}
# sample_weights = np.array([class_weights_dict[y] for y in y_train_RNN_noPOS])

# model_class_norm_lstm = Sequential()
# model_class_norm_lstm.add(Embedding(input_dim=len(tokenizer_RNN_noPOS.word_index) + 1, output_dim=128, input_length=10))
# model_class_norm_lstm.add(LSTM(units=50))
# model_class_norm_lstm.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), input_dim=(10,1)))
# model_class_norm_lstm.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
# model_class_norm_lstm.add(Dense(3, activation='softmax'))

# model_class_norm_lstm.compile(
#     loss='sparse_categorical_crossentropy', 
#     optimizer='adam', 
#     metrics=['accuracy'],    
#     weighted_metrics=['accuracy']
# )

# model_class_norm_lstm.fit(X_train_RNN_noPOS, y_train_RNN_noPOS, epochs=10, batch_size=64, validation_split=0.2, sample_weight=sample_weights)

# # test predictions and get evaluation metrics
# pred_noPOS_class_norm_lstm = model_class_norm_lstm.predict(X_test_RNN_noPOS)

# #convert back
# pred_noPOS_class_norm_lstm_converted = np.argmax(pred_noPOS_class_norm_lstm, axis=1)

# #evaluate results from the RNN
# class_norm_lstm_report_noPOS = classification_report(y_test_RNN_noPOS, pred_noPOS_class_norm_lstm_converted)
# class_norm_lstm_confusion_matrix_noPOS = confusion_matrix(y_test_RNN_noPOS, pred_noPOS_class_norm_lstm_converted)
# class_norm_lstm_accuracy_noPOS = accuracy_score(y_test_RNN_noPOS, pred_noPOS_class_norm_lstm_converted)

# #print the results
# print("Classification Report:\n", class_norm_lstm_report_noPOS)
# print("Confusion Matrix:\n", class_norm_lstm_confusion_matrix_noPOS)
# print("Accuracy:", class_norm_lstm_accuracy_noPOS)

Epoch 1/10
5158/5158 [==============================] - 85s 16ms/step - loss: 0.8780 - accuracy: 0.7258 - weighted_accuracy: 0.5839 - val_loss: 0.8343 - val_accuracy: 0.7262 - val_weighted_accuracy: 0.6120
Epoch 2/10
5158/5158 [==============================] - 85s 16ms/step - loss: 0.7674 - accuracy: 0.7534 - weighted_accuracy: 0.6483 - val_loss: 0.8231 - val_accuracy: 0.7570 - val_weighted_accuracy: 0.6123
Epoch 3/10
5158/5158 [==============================] - 85s 16ms/step - loss: 0.7039 - accuracy: 0.7697 - weighted_accuracy: 0.6922 - val_loss: 0.8694 - val_accuracy: 0.7282 - val_weighted_accuracy: 0.6062
Epoch 4/10
5158/5158 [==============================] - 85s 16ms/step - loss: 0.6390 - accuracy: 0.7866 - weighted_accuracy: 0.7342 - val_loss: 0.9227 - val_accuracy: 0.7675 - val_weighted_accuracy: 0.5998
Epoch 5/10
5158/5158 [==============================] - 85s 16ms/step - loss: 0.5724 - accuracy: 0.8001 - weighted_accuracy: 0.7719 - val_loss: 0.9632 - val_accuracy: 0.7437 - 

#### 7.4.8: Same as above, but with L2 normalization first, then LSTM 

In [34]:
# # next I will try a new model architecture with class weights adjusted and L2 norm and LSTM
# class_weights_lstm = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_RNN_noPOS), y=y_train_RNN_noPOS)
# class_weights_dict = {i: weight for i, weight in enumerate(class_weights_lstm)}
# sample_weights = np.array([class_weights_dict[y] for y in y_train_RNN_noPOS])

# model_class_norm_lstm_2 = Sequential()
# model_class_norm_lstm_2.add(Embedding(input_dim=len(tokenizer_RNN_noPOS.word_index) + 1, output_dim=128, input_length=10))
# model_class_norm_lstm_2.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), input_dim=(10,1)))
# model_class_norm_lstm_2.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
# model_class_norm_lstm_2.add(LSTM(units=50))
# model_class_norm_lstm_2.add(Dense(3, activation='softmax'))

# model_class_norm_lstm_2.compile(
#     loss='sparse_categorical_crossentropy', 
#     optimizer='adam', 
#     metrics=['accuracy'],    
#     weighted_metrics=['accuracy']
# )

# model_class_norm_lstm_2.fit(X_train_RNN_noPOS, y_train_RNN_noPOS, epochs=10, batch_size=64, validation_split=0.2, sample_weight=sample_weights)

# # test predictions and get evaluation metrics
# pred_noPOS_class_norm_lstm_2 = model_class_norm_lstm_2.predict(X_test_RNN_noPOS)

# #convert back
# pred_noPOS_class_norm_lstm_converted_2 = np.argmax(pred_noPOS_class_norm_lstm_2, axis=1)

# #evaluate results from the RNN
# class_norm_lstm_report_noPOS_2 = classification_report(y_test_RNN_noPOS, pred_noPOS_class_norm_lstm_converted_2)
# class_norm_lstm_confusion_matrix_noPOS_2 = confusion_matrix(y_test_RNN_noPOS, pred_noPOS_class_norm_lstm_converted_2)
# class_norm_lstm_accuracy_noPOS_2 = accuracy_score(y_test_RNN_noPOS, pred_noPOS_class_norm_lstm_converted_2)

# #print the results
# print("Classification Report:\n", class_norm_lstm_report_noPOS_2)
# print("Confusion Matrix:\n", class_norm_lstm_confusion_matrix_noPOS_2)
# print("Accuracy:", class_norm_lstm_accuracy_noPOS_2)

Epoch 1/10
5158/5158 [==============================] - 83s 16ms/step - loss: 0.8989 - accuracy: 0.6972 - weighted_accuracy: 0.5672 - val_loss: 0.8685 - val_accuracy: 0.6545 - val_weighted_accuracy: 0.5816
Epoch 2/10
5158/5158 [==============================] - 82s 16ms/step - loss: 0.8167 - accuracy: 0.7222 - weighted_accuracy: 0.6123 - val_loss: 0.8542 - val_accuracy: 0.7322 - val_weighted_accuracy: 0.5914
Epoch 3/10
5158/5158 [==============================] - 82s 16ms/step - loss: 0.7926 - accuracy: 0.7250 - weighted_accuracy: 0.6284 - val_loss: 0.8712 - val_accuracy: 0.7196 - val_weighted_accuracy: 0.5833
Epoch 4/10
5158/5158 [==============================] - 82s 16ms/step - loss: 0.7769 - accuracy: 0.7260 - weighted_accuracy: 0.6414 - val_loss: 0.8826 - val_accuracy: 0.6351 - val_weighted_accuracy: 0.5805
Epoch 5/10
5158/5158 [==============================] - 82s 16ms/step - loss: 0.7663 - accuracy: 0.7260 - weighted_accuracy: 0.6490 - val_loss: 0.9020 - val_accuracy: 0.7398 - 

#### 7.4.9: Trying class weights with Bi-directional LSTM

In [19]:
# # next I will try a new model architecture with class weights adjusted and L2 norm and Bi-Directional LSTM
# class_weights_bi_lstm = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_RNN_noPOS), y=y_train_RNN_noPOS)
# class_weights_dict = {i: weight for i, weight in enumerate(class_weights_bi_lstm)}
# sample_weights = np.array([class_weights_dict[y] for y in y_train_RNN_noPOS])

# model_class_norm_lstm_bi = Sequential()
# model_class_norm_lstm_bi.add(Embedding(input_dim=len(tokenizer_RNN_noPOS.word_index) + 1, output_dim=128, input_length=10))
# model_class_norm_lstm_bi.add(Bidirectional(LSTM(units=50, kernel_regularizer=l2(0.01))))
# model_class_norm_lstm_bi.add(Dense(3, activation='softmax'))

# model_class_norm_lstm_bi.compile(
#     loss='sparse_categorical_crossentropy', 
#     optimizer='adam', 
#     metrics=['accuracy'],    
#     weighted_metrics=['accuracy']
# )

# model_class_norm_lstm_bi.fit(X_train_RNN_noPOS, y_train_RNN_noPOS, epochs=10, batch_size=64, validation_split=0.2, sample_weight=sample_weights)

# # test predictions and get evaluation metrics
# pred_noPOS_class_norm_lstm_bi = model_class_norm_lstm_bi.predict(X_test_RNN_noPOS)

# #convert back
# pred_noPOS_class_norm_lstm_converted_bi = np.argmax(pred_noPOS_class_norm_lstm_bi, axis=1)

# #evaluate results from the RNN
# class_norm_lstm_report_noPOS_bi = classification_report(y_test_RNN_noPOS, pred_noPOS_class_norm_lstm_converted_bi)
# class_norm_lstm_confusion_matrix_noPOS_bi = confusion_matrix(y_test_RNN_noPOS, pred_noPOS_class_norm_lstm_converted_bi)
# class_norm_lstm_accuracy_noPOS_bi = accuracy_score(y_test_RNN_noPOS, pred_noPOS_class_norm_lstm_converted_bi)

# #print the results
# print("Classification Report:\n", class_norm_lstm_report_noPOS_bi)
# print("Confusion Matrix:\n", class_norm_lstm_confusion_matrix_noPOS_bi)
# print("Accuracy:", class_norm_lstm_accuracy_noPOS_bi)

Epoch 1/10
5158/5158 [==============================] - 90s 17ms/step - loss: 0.8972 - accuracy: 0.7156 - weighted_accuracy: 0.5758 - val_loss: 0.8563 - val_accuracy: 0.6829 - val_weighted_accuracy: 0.5993
Epoch 2/10
5158/5158 [==============================] - 89s 17ms/step - loss: 0.7987 - accuracy: 0.7322 - weighted_accuracy: 0.6335 - val_loss: 0.8720 - val_accuracy: 0.7885 - val_weighted_accuracy: 0.5906
Epoch 3/10
5158/5158 [==============================] - 89s 17ms/step - loss: 0.7614 - accuracy: 0.7387 - weighted_accuracy: 0.6613 - val_loss: 0.9019 - val_accuracy: 0.7712 - val_weighted_accuracy: 0.5893
Epoch 4/10
5158/5158 [==============================] - 89s 17ms/step - loss: 0.7309 - accuracy: 0.7478 - weighted_accuracy: 0.6852 - val_loss: 0.8846 - val_accuracy: 0.6401 - val_weighted_accuracy: 0.5839
Epoch 5/10
5158/5158 [==============================] - 91s 18ms/step - loss: 0.7051 - accuracy: 0.7545 - weighted_accuracy: 0.7037 - val_loss: 0.9064 - val_accuracy: 0.7665 - 

#### 7.4.10: Trying with same as above but GRU instead 

In [20]:
# # next I will try a new model architecture with class weights adjusted and L2 norm and GRU
# class_weights_gru = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_RNN_noPOS), y=y_train_RNN_noPOS)
# class_weights_dict = {i: weight for i, weight in enumerate(class_weights_gru)}
# sample_weights = np.array([class_weights_dict[y] for y in y_train_RNN_noPOS])

# model_class_norm_gru = Sequential()
# model_class_norm_gru.add(Embedding(input_dim=len(tokenizer_RNN_noPOS.word_index) + 1, output_dim=128, input_length=10))
# model_class_norm_gru.add(GRU(units=50, kernel_regularizer=l2(0.01)))
# model_class_norm_gru.add(Dense(3, activation='softmax'))

# model_class_norm_gru.compile(
#     loss='sparse_categorical_crossentropy', 
#     optimizer='adam', 
#     metrics=['accuracy'],    
#     weighted_metrics=['accuracy']
# )

# model_class_norm_gru.fit(X_train_RNN_noPOS, y_train_RNN_noPOS, epochs=10, batch_size=64, validation_split=0.2, sample_weight=sample_weights)

# # test predictions and get evaluation metrics
# pred_noPOS_class_norm_gru = model_class_norm_gru.predict(X_test_RNN_noPOS)

# #convert back
# pred_noPOS_class_norm_converted_gru = np.argmax(pred_noPOS_class_norm_gru, axis=1)

# #evaluate results from the RNN
# class_norm_lstm_report_noPOS_gru = classification_report(y_test_RNN_noPOS, pred_noPOS_class_norm_converted_gru)
# class_norm_lstm_confusion_matrix_noPOS_gru = confusion_matrix(y_test_RNN_noPOS, pred_noPOS_class_norm_converted_gru)
# class_norm_lstm_accuracy_noPOS_gru = accuracy_score(y_test_RNN_noPOS, pred_noPOS_class_norm_converted_gru)

# #print the results
# print("Classification Report:\n", class_norm_lstm_report_noPOS_gru)
# print("Confusion Matrix:\n", class_norm_lstm_confusion_matrix_noPOS_gru)
# print("Accuracy:", class_norm_lstm_accuracy_noPOS_gru)

Epoch 1/10
5158/5158 [==============================] - 83s 16ms/step - loss: 0.8827 - accuracy: 0.7136 - weighted_accuracy: 0.5835 - val_loss: 0.8475 - val_accuracy: 0.7263 - val_weighted_accuracy: 0.6013
Epoch 2/10
5158/5158 [==============================] - 84s 16ms/step - loss: 0.7952 - accuracy: 0.7337 - weighted_accuracy: 0.6366 - val_loss: 0.8451 - val_accuracy: 0.7173 - val_weighted_accuracy: 0.6013
Epoch 3/10
5158/5158 [==============================] - 84s 16ms/step - loss: 0.7610 - accuracy: 0.7403 - weighted_accuracy: 0.6602 - val_loss: 0.8743 - val_accuracy: 0.6316 - val_weighted_accuracy: 0.5850
Epoch 4/10
5158/5158 [==============================] - 86s 17ms/step - loss: 0.7335 - accuracy: 0.7475 - weighted_accuracy: 0.6788 - val_loss: 0.8945 - val_accuracy: 0.7447 - val_weighted_accuracy: 0.5909
Epoch 5/10
5158/5158 [==============================] - 86s 17ms/step - loss: 0.7107 - accuracy: 0.7541 - weighted_accuracy: 0.6915 - val_loss: 0.8988 - val_accuracy: 0.7439 - 

#### 7.4.11: SimpleRNN with class weights balanced and L2 Normalization

In [22]:
# #adjust class weights than train with SimpleRNN
# class_weights_simple = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_RNN_noPOS), y=y_train_RNN_noPOS)
# class_weights_dict = {i: weight for i, weight in enumerate(class_weights_simple)}
# sample_weights = np.array([class_weights_dict[y] for y in y_train_RNN_noPOS])

# model_simpleRNN_normalized_class = Sequential()
# model_simpleRNN_normalized_class.add(Embedding(input_dim=len(tokenizer_RNN_noPOS.word_index) + 1, output_dim=128, input_length=10))
# model_simpleRNN_normalized_class.add(SimpleRNN(units=50))
# model_simpleRNN_normalized_class.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), input_dim=(10,1)))
# model_simpleRNN_normalized_class.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
# model_simpleRNN_normalized_class.add(Dense(3, activation='softmax'))

# model_simpleRNN_normalized_class.compile(
#     loss='sparse_categorical_crossentropy', 
#     optimizer='adam', 
#     metrics=['accuracy'],    
#     weighted_metrics=['accuracy']
# )

# model_simpleRNN_normalized_class.fit(X_train_RNN_noPOS, y_train_RNN_noPOS, epochs=10, batch_size=64, validation_split=0.2, sample_weight=sample_weights)

# # test predictions and get evaluation metrics
# pred_simpleRNN_noPOS_normalized_class = model_simpleRNN_normalized_class.predict(X_test_RNN_noPOS)

# #convert back
# pred_simpleRNN_noPOS_normalized_class_convert = np.argmax(pred_simpleRNN_noPOS_normalized_class, axis=1)

# #evaluate results from the RNN
# only_normalized_simpleRNN_class_report_noPOS = classification_report(y_test_RNN_noPOS, pred_simpleRNN_noPOS_normalized_class_convert)
# only_normalized_simpleRNN_confusion_matrix_class_noPOS = confusion_matrix(y_test_RNN_noPOS, pred_simpleRNN_noPOS_normalized_class_convert)
# only_normalized_simpleRNN_accuracy_class_noPOS = accuracy_score(y_test_RNN_noPOS, pred_simpleRNN_noPOS_normalized_class_convert)

# #print the results
# print("Classification Report:\n", only_normalized_simpleRNN_class_report_noPOS)
# print("Confusion Matrix:\n", only_normalized_simpleRNN_confusion_matrix_class_noPOS)
# print("Accuracy:", only_normalized_simpleRNN_accuracy_class_noPOS)

Epoch 1/10
5158/5158 [==============================] - 80s 15ms/step - loss: 0.9035 - accuracy: 0.7206 - weighted_accuracy: 0.5741 - val_loss: 0.8568 - val_accuracy: 0.7485 - val_weighted_accuracy: 0.5987
Epoch 2/10
5158/5158 [==============================] - 81s 16ms/step - loss: 0.7866 - accuracy: 0.7380 - weighted_accuracy: 0.6433 - val_loss: 0.8660 - val_accuracy: 0.7551 - val_weighted_accuracy: 0.5946
Epoch 3/10
5158/5158 [==============================] - 83s 16ms/step - loss: 0.7037 - accuracy: 0.7594 - weighted_accuracy: 0.7017 - val_loss: 0.8966 - val_accuracy: 0.6541 - val_weighted_accuracy: 0.5846
Epoch 4/10
5158/5158 [==============================] - 82s 16ms/step - loss: 0.6205 - accuracy: 0.7743 - weighted_accuracy: 0.7522 - val_loss: 0.9541 - val_accuracy: 0.7804 - val_weighted_accuracy: 0.5808
Epoch 5/10
5158/5158 [==============================] - 81s 16ms/step - loss: 0.5472 - accuracy: 0.7894 - weighted_accuracy: 0.7902 - val_loss: 1.0117 - val_accuracy: 0.7044 - 

## Part 8: Updated Classification Data to only 2 classes - positive and negative 

In [20]:
#import TFIDF for testing using this data instead of current features
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
# first I will define the thresholds - our labels will be positive, negative, and neutral 
# I decided to use a small window for neutral, between 4.5 and 5.5 - this will ensure most the review classifications
# are sensative - it can be updated later if desired by changing the following threshold values

positive_threshold = 8.0
negative_threshold = 8.0

# next, I will classify by building a classification function

def classify_scores(score_value):
    if score_value >= positive_threshold:
        return 'positive'
    elif score_value < negative_threshold:
        return 'negative'
    
# I will also do the same for the no POS tagging scenario

preprocessed_reviews_updated = preprocessed_reviews_no_pos_tagging.copy()
    
preprocessed_reviews_updated['Reviewer_Score'] = preprocessed_reviews_updated['Reviewer_Score'].apply(classify_scores)

preprocessed_reviews_updated

# determine the counts of each category
review_counts_updated = preprocessed_reviews_updated['Reviewer_Score'].value_counts()
print(review_counts_updated)

positive    335646
negative    180092
Name: Reviewer_Score, dtype: int64


In [46]:
#with an RNN the scores can not be text - they need to be numbers - will encode these now
class LabelEncoderSentiment_updated:
    def __init__(self):
        self.classes_ = ['negative', 'positive']

    def fit_transform_updated(self, labels):
        return np.array([self.classes_.index(label) for label in labels])
    
    def inverse_transform_sentiment_updated(encoded_labels):
        classes_ = ['negative', 'positive']
        return [classes_[label] for label in encoded_labels]

label_encoder_RNN_updated = LabelEncoderSentiment_updated()
encoded_scores_RNN_updated = label_encoder_RNN_updated.fit_transform_updated(preprocessed_reviews_updated["Reviewer_Score"])

# get the text and POS data
review_RNN_updated = preprocessed_reviews_updated["Total_Review_handled"]

print(preprocessed_reviews_updated)

# # Define TF-IDF vectorizer - this didn't work well so I won't use it. 
# tfidf_vectorizer = TfidfVectorizer(max_features=500)
# preprocessed_reviews_tfidf = [" ".join(seq) for seq in review_RNN_updated]
# # Fit and transform the text data using TF-IDF
# tfidf_matrix = tfidf_vectorizer.fit_transform(preprocessed_reviews_tfidf)
# tfidf_array = tfidf_matrix.toarray()

# prepare the text and POS data for RNN usage
tokenizer_RNN_updated = Tokenizer()
tokenizer_RNN_updated.fit_on_texts([" ".join(seq) for seq in review_RNN_updated])
sequences_RNN_updated = tokenizer_RNN_updated.texts_to_sequences([" ".join(seq) for seq in review_RNN_updated])
review_prepped_RNN_updated = pad_sequences(sequences_RNN_updated, maxlen=30)

# prepare the test and train datasets
X_train_RNN_updated, X_test_RNN_updated, y_train_RNN_updated, y_test_RNN_updated = train_test_split(review_prepped_RNN_updated, encoded_scores_RNN_updated, test_size=0.2, random_state=42)

                                     Total_Review_handled Reviewer_Score
0       [angry, made, post, available, via, possible, ...       negative
1       [negative, real, complaint, hotel, great, grea...       negative
2       [room, nice, elderly, bit, difficult, room, tw...       negative
3       [room, dirty, afraid, walk, barefoot, floor, l...       negative
4       [booked, company, line, showed, picture, room,...       negative
...                                                   ...            ...
515733  [trolly, staff, help, take, luggage, room, loc...       negative
515734  [hotel, look, like, surely, breakfast, ok, got...       negative
515735  [ac, useless, hot, week, vienna, gave, hot, ai...       negative
515736  [negative, room, enormous, really, comfortable...       positive
515737         [rd, floor, work, free, wife, staff, kind]       positive

[515738 rows x 2 columns]


In [23]:
#test batch normalization and reducing the learning rate if no changes to val_loss

from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau

#update class weights
class_weights_updated = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_RNN_updated), y=y_train_RNN_updated)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights_updated)}
sample_weights_updated = np.array([class_weights_dict[y] for y in y_train_RNN_updated])

#define the reduce learning rate parameters. 
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  
    factor=0.5,           
    patience=3,           
    min_lr=1e-5,          
    verbose=1              
)

In [24]:
#create the complex model
model_simpleRNN_updated = Sequential()
model_simpleRNN_updated.add(Embedding(input_dim=len(tokenizer_RNN_updated.word_index) + 1, output_dim=128, input_length=30))
model_simpleRNN_updated.add(BatchNormalization())
model_simpleRNN_updated.add(SimpleRNN(units=50))
model_simpleRNN_updated.add(BatchNormalization())
model_simpleRNN_updated.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model_simpleRNN_updated.add(BatchNormalization())
model_simpleRNN_updated.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
model_simpleRNN_updated.add(BatchNormalization())
model_simpleRNN_updated.add(Dense(2, activation='softmax'))

model_simpleRNN_updated.compile(
    loss='sparse_categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy'],    
    weighted_metrics=['accuracy']
)

#train
model_simpleRNN_updated.fit(X_train_RNN_updated, y_train_RNN_updated, epochs=100, batch_size=64, validation_split=0.2, sample_weight=sample_weights_updated, callbacks=[reduce_lr])

# test predictions and get evaluation metrics
pred_simpleRNN_updated = model_simpleRNN_updated.predict(X_test_RNN_updated)

#convert back
pred_simpleRNN_updated_convert = np.argmax(pred_simpleRNN_updated, axis=1)

#evaluate results from the RNN
updated_simpleRNN_class_report = classification_report(y_test_RNN_updated, pred_simpleRNN_updated_convert)
updated_simpleRNN_confusion_matrix_class = confusion_matrix(y_test_RNN_updated, pred_simpleRNN_updated_convert)
updated_simpleRNN_accuracy_class = accuracy_score(y_test_RNN_updated, pred_simpleRNN_updated_convert)

#print the results
print("Classification Report:\n", updated_simpleRNN_class_report)
print("Confusion Matrix:\n", updated_simpleRNN_confusion_matrix_class)
print("Accuracy:", updated_simpleRNN_accuracy_class)

Epoch 1/100
5158/5158 [==============================] - 85s 16ms/step - loss: 0.6525 - accuracy: 0.6990 - weighted_accuracy: 0.6998 - val_loss: 0.5716 - val_accuracy: 0.7224 - val_weighted_accuracy: 0.7130 - lr: 0.0010
Epoch 2/100
5158/5158 [==============================] - 85s 16ms/step - loss: 0.5133 - accuracy: 0.7483 - weighted_accuracy: 0.7518 - val_loss: 0.5202 - val_accuracy: 0.7567 - val_weighted_accuracy: 0.7494 - lr: 0.0010
Epoch 3/100
5158/5158 [==============================] - 85s 16ms/step - loss: 0.4881 - accuracy: 0.7683 - weighted_accuracy: 0.7684 - val_loss: 0.5871 - val_accuracy: 0.7184 - val_weighted_accuracy: 0.7218 - lr: 0.0010
Epoch 4/100
5158/5158 [==============================] - 85s 16ms/step - loss: 0.5125 - accuracy: 0.7477 - weighted_accuracy: 0.7541 - val_loss: 0.5149 - val_accuracy: 0.7610 - val_weighted_accuracy: 0.7570 - lr: 0.0010
Epoch 5/100
5158/5158 [==============================] - 85s 16ms/step - loss: 0.4908 - accuracy: 0.7673 - weighted_accu

Epoch 32/100
5158/5158 [==============================] - 86s 17ms/step - loss: 0.3202 - accuracy: 0.8646 - weighted_accuracy: 0.8622 - val_loss: 0.5423 - val_accuracy: 0.7719 - val_weighted_accuracy: 0.7655 - lr: 1.0000e-05
Epoch 33/100
5158/5158 [==============================] - 85s 17ms/step - loss: 0.3194 - accuracy: 0.8652 - weighted_accuracy: 0.8631 - val_loss: 0.5421 - val_accuracy: 0.7708 - val_weighted_accuracy: 0.7658 - lr: 1.0000e-05
Epoch 34/100
5158/5158 [==============================] - 86s 17ms/step - loss: 0.3195 - accuracy: 0.8652 - weighted_accuracy: 0.8630 - val_loss: 0.5433 - val_accuracy: 0.7726 - val_weighted_accuracy: 0.7655 - lr: 1.0000e-05
Epoch 35/100
5158/5158 [==============================] - 86s 17ms/step - loss: 0.3195 - accuracy: 0.8651 - weighted_accuracy: 0.8628 - val_loss: 0.5434 - val_accuracy: 0.7715 - val_weighted_accuracy: 0.7654 - lr: 1.0000e-05
Epoch 36/100
5158/5158 [==============================] - 86s 17ms/step - loss: 0.3191 - accuracy: 0

5158/5158 [==============================] - 86s 17ms/step - loss: 0.3108 - accuracy: 0.8689 - weighted_accuracy: 0.8671 - val_loss: 0.5552 - val_accuracy: 0.7705 - val_weighted_accuracy: 0.7638 - lr: 1.0000e-05
Epoch 69/100
5158/5158 [==============================] - 85s 17ms/step - loss: 0.3103 - accuracy: 0.8694 - weighted_accuracy: 0.8676 - val_loss: 0.5532 - val_accuracy: 0.7707 - val_weighted_accuracy: 0.7638 - lr: 1.0000e-05
Epoch 70/100
5158/5158 [==============================] - 85s 17ms/step - loss: 0.3095 - accuracy: 0.8696 - weighted_accuracy: 0.8678 - val_loss: 0.5584 - val_accuracy: 0.7696 - val_weighted_accuracy: 0.7635 - lr: 1.0000e-05
Epoch 71/100
5158/5158 [==============================] - 85s 17ms/step - loss: 0.3100 - accuracy: 0.8697 - weighted_accuracy: 0.8677 - val_loss: 0.5558 - val_accuracy: 0.7722 - val_weighted_accuracy: 0.7637 - lr: 1.0000e-05
Epoch 72/100
5158/5158 [==============================] - 85s 17ms/step - loss: 0.3098 - accuracy: 0.8698 - weigh

In [40]:
#create the model
model_lstm_updated = Sequential()
model_lstm_updated.add(Embedding(input_dim=len(tokenizer_RNN_updated.word_index) + 1, output_dim=128, input_length=30))
model_lstm_updated.add(BatchNormalization())
model_lstm_updated.add(LSTM(units=50, kernel_regularizer=l2(0.1)))
model_lstm_updated.add(BatchNormalization())
model_lstm_updated.add(Dense(2, activation='softmax'))

model_lstm_updated.compile(
    loss='sparse_categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy'],    
    weighted_metrics=['accuracy']
)

#train
model_lstm_updated.fit(X_train_RNN_updated, y_train_RNN_updated, epochs=100, batch_size=64, validation_split=0.2, sample_weight=sample_weights_updated, callbacks=[reduce_lr])

# test predictions and get evaluation metrics
pred_lstm_updated = model_lstm_updated.predict(X_test_RNN_updated)

#convert back
pred_lstm_updated_convert = np.argmax(pred_lstm_updated, axis=1)

#evaluate results from the RNN
updated_lstm_class_report = classification_report(y_test_RNN_updated, pred_lstm_updated_convert)
updated_lstm_confusion_matrix_class = confusion_matrix(y_test_RNN_updated, pred_lstm_updated_convert)
updated_lstm_accuracy_class = accuracy_score(y_test_RNN_updated, pred_lstm_updated_convert)

#print the results
print("Classification Report:\n", updated_lstm_class_report)
print("Confusion Matrix:\n", updated_lstm_confusion_matrix_class)
print("Accuracy:", updated_lstm_accuracy_class)

Epoch 1/100
5158/5158 [==============================] - 124s 24ms/step - loss: 0.6472 - accuracy: 0.7600 - weighted_accuracy: 0.7630 - val_loss: 0.4884 - val_accuracy: 0.7850 - val_weighted_accuracy: 0.7728 - lr: 0.0010
Epoch 2/100
5158/5158 [==============================] - 122s 24ms/step - loss: 0.4646 - accuracy: 0.7858 - weighted_accuracy: 0.7872 - val_loss: 0.4734 - val_accuracy: 0.7688 - val_weighted_accuracy: 0.7800 - lr: 0.0010
Epoch 3/100
5158/5158 [==============================] - 122s 24ms/step - loss: 0.4454 - accuracy: 0.7958 - weighted_accuracy: 0.7970 - val_loss: 0.4756 - val_accuracy: 0.7882 - val_weighted_accuracy: 0.7795 - lr: 0.0010
Epoch 4/100
5158/5158 [==============================] - 121s 23ms/step - loss: 0.4313 - accuracy: 0.8037 - weighted_accuracy: 0.8047 - val_loss: 0.4728 - val_accuracy: 0.7763 - val_weighted_accuracy: 0.7819 - lr: 0.0010
Epoch 5/100
5158/5158 [==============================] - 121s 24ms/step - loss: 0.4204 - accuracy: 0.8091 - weighted

5158/5158 [==============================] - 122s 24ms/step - loss: 0.3125 - accuracy: 0.8581 - weighted_accuracy: 0.8586 - val_loss: 0.6463 - val_accuracy: 0.7633 - val_weighted_accuracy: 0.7591 - lr: 1.0000e-05
Epoch 68/100
5158/5158 [==============================] - 121s 24ms/step - loss: 0.3125 - accuracy: 0.8578 - weighted_accuracy: 0.8585 - val_loss: 0.6438 - val_accuracy: 0.7612 - val_weighted_accuracy: 0.7593 - lr: 1.0000e-05
Epoch 69/100
5158/5158 [==============================] - 121s 24ms/step - loss: 0.3119 - accuracy: 0.8579 - weighted_accuracy: 0.8587 - val_loss: 0.6452 - val_accuracy: 0.7637 - val_weighted_accuracy: 0.7598 - lr: 1.0000e-05
Epoch 70/100
5158/5158 [==============================] - 121s 24ms/step - loss: 0.3119 - accuracy: 0.8577 - weighted_accuracy: 0.8584 - val_loss: 0.6468 - val_accuracy: 0.7626 - val_weighted_accuracy: 0.7599 - lr: 1.0000e-05
Epoch 71/100
5158/5158 [==============================] - 122s 24ms/step - loss: 0.3120 - accuracy: 0.8576 - 

In [41]:
#create the bi-lstm model
model_bi_lstm_updated = Sequential()
model_bi_lstm_updated.add(Embedding(input_dim=len(tokenizer_RNN_updated.word_index) + 1, output_dim=128, input_length=30))
model_bi_lstm_updated.add(BatchNormalization())
model_bi_lstm_updated.add(Bidirectional(LSTM(units=50, kernel_regularizer=l2(0.1))))
model_bi_lstm_updated.add(BatchNormalization())
model_bi_lstm_updated.add(Dense(2, activation='softmax'))

model_bi_lstm_updated.compile(
    loss='sparse_categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy'],    
    weighted_metrics=['accuracy']
)

#train
model_bi_lstm_updated.fit(X_train_RNN_updated, y_train_RNN_updated, epochs=100, batch_size=64, validation_split=0.2, sample_weight=sample_weights_updated, callbacks=[reduce_lr])

# test predictions and get evaluation metrics
pred_bi_lstm_updated = model_bi_lstm_updated.predict(X_test_RNN_updated)

#convert back
pred_bi_lstm_updated_convert = np.argmax(pred_bi_lstm_updated, axis=1)

#evaluate results from the RNN
updated_bi_lstm_class_report = classification_report(y_test_RNN_updated, pred_bi_lstm_updated_convert)
updated_bi_lstm_confusion_matrix_class = confusion_matrix(y_test_RNN_updated, pred_bi_lstm_updated_convert)
updated_bi_lstm_accuracy_class = accuracy_score(y_test_RNN_updated, pred_bi_lstm_updated_convert)

#print the results
print("Classification Report:\n", updated_bi_lstm_class_report)
print("Confusion Matrix:\n", updated_bi_lstm_confusion_matrix_class)
print("Accuracy:", updated_bi_lstm_accuracy_class)

Epoch 1/100
5158/5158 [==============================] - 135s 26ms/step - loss: 0.7661 - accuracy: 0.7613 - weighted_accuracy: 0.7643 - val_loss: 0.4911 - val_accuracy: 0.7488 - val_weighted_accuracy: 0.7732 - lr: 0.0010
Epoch 2/100
5158/5158 [==============================] - 134s 26ms/step - loss: 0.4693 - accuracy: 0.7856 - weighted_accuracy: 0.7873 - val_loss: 0.4827 - val_accuracy: 0.7688 - val_weighted_accuracy: 0.7759 - lr: 0.0010
Epoch 3/100
5158/5158 [==============================] - 134s 26ms/step - loss: 0.4508 - accuracy: 0.7939 - weighted_accuracy: 0.7951 - val_loss: 0.4994 - val_accuracy: 0.7850 - val_weighted_accuracy: 0.7671 - lr: 0.0010
Epoch 4/100
5158/5158 [==============================] - 133s 26ms/step - loss: 0.4333 - accuracy: 0.8018 - weighted_accuracy: 0.8028 - val_loss: 0.4773 - val_accuracy: 0.7812 - val_weighted_accuracy: 0.7784 - lr: 0.0010
Epoch 5/100
5158/5158 [==============================] - 133s 26ms/step - loss: 0.4228 - accuracy: 0.8073 - weighted

5158/5158 [==============================] - 134s 26ms/step - loss: 0.3173 - accuracy: 0.8562 - weighted_accuracy: 0.8565 - val_loss: 0.6300 - val_accuracy: 0.7656 - val_weighted_accuracy: 0.7592 - lr: 1.0000e-05
Epoch 68/100
5158/5158 [==============================] - 134s 26ms/step - loss: 0.3173 - accuracy: 0.8561 - weighted_accuracy: 0.8562 - val_loss: 0.6299 - val_accuracy: 0.7632 - val_weighted_accuracy: 0.7595 - lr: 1.0000e-05
Epoch 69/100
5158/5158 [==============================] - 134s 26ms/step - loss: 0.3169 - accuracy: 0.8561 - weighted_accuracy: 0.8566 - val_loss: 0.6264 - val_accuracy: 0.7637 - val_weighted_accuracy: 0.7596 - lr: 1.0000e-05
Epoch 70/100
5158/5158 [==============================] - 134s 26ms/step - loss: 0.3172 - accuracy: 0.8561 - weighted_accuracy: 0.8564 - val_loss: 0.6279 - val_accuracy: 0.7632 - val_weighted_accuracy: 0.7595 - lr: 1.0000e-05
Epoch 71/100
5158/5158 [==============================] - 134s 26ms/step - loss: 0.3167 - accuracy: 0.8563 - 

In [24]:
# create the GRU model
model_gru_updated = Sequential()
model_gru_updated.add(Embedding(input_dim=len(tokenizer_RNN_updated.word_index) + 1, output_dim=128, input_length=30))
model_gru_updated.add(BatchNormalization())
model_gru_updated.add(GRU(units=100, kernel_regularizer=l2(0.01), return_sequences=True))
model_gru_updated.add(BatchNormalization())
model_gru_updated.add(GRU(units=50, kernel_regularizer=l2(0.01)))
model_gru_updated.add(BatchNormalization())
model_gru_updated.add(Dropout(0.5))
model_gru_updated.add(Dense(2, activation='softmax'))

model_gru_updated.compile(
    loss='sparse_categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy'],    
    weighted_metrics=['accuracy']
)

#train
model_gru_updated.fit(X_train_RNN_updated, y_train_RNN_updated, epochs=10, batch_size=64, validation_split=0.2, sample_weight=sample_weights_updated, callbacks=[reduce_lr])

# test predictions and get evaluation metrics
pred_gru_updated = model_gru_updated.predict(X_test_RNN_updated)

#convert back
pred_gru_updated_convert = np.argmax(pred_gru_updated, axis=1)

#evaluate results from the RNN
updated_gru_class_report = classification_report(y_test_RNN_updated, pred_gru_updated_convert)
updated_gru_confusion_matrix_class = confusion_matrix(y_test_RNN_updated, pred_gru_updated_convert)
updated_gru_accuracy_class = accuracy_score(y_test_RNN_updated, pred_gru_updated_convert)

#print the results
print("Classification Report:\n", updated_gru_class_report)
print("Confusion Matrix:\n", updated_gru_confusion_matrix_class)
print("Accuracy:", updated_gru_accuracy_class)

Epoch 1/10
5158/5158 [==============================] - 220s 42ms/step - loss: 0.6046 - accuracy: 0.7618 - weighted_accuracy: 0.7645 - val_loss: 0.4736 - val_accuracy: 0.7823 - val_weighted_accuracy: 0.7763 - lr: 0.0010
Epoch 2/10
5158/5158 [==============================] - 221s 43ms/step - loss: 0.4616 - accuracy: 0.7888 - weighted_accuracy: 0.7896 - val_loss: 0.4822 - val_accuracy: 0.7942 - val_weighted_accuracy: 0.7773 - lr: 0.0010
Epoch 3/10
5158/5158 [==============================] - 234s 45ms/step - loss: 0.4429 - accuracy: 0.7994 - weighted_accuracy: 0.7996 - val_loss: 0.4679 - val_accuracy: 0.7811 - val_weighted_accuracy: 0.7815 - lr: 0.0010
Epoch 4/10
5158/5158 [==============================] - 248s 48ms/step - loss: 0.4280 - accuracy: 0.8087 - weighted_accuracy: 0.8085 - val_loss: 0.4845 - val_accuracy: 0.7880 - val_weighted_accuracy: 0.7729 - lr: 0.0010
Epoch 5/10
5158/5158 [==============================] - 242s 47ms/step - loss: 0.4140 - accuracy: 0.8169 - weighted_accu

Taking the best results with the SimpleRNN, I will test a larger batch size as a scenario to see if that helps. I will also try to overfit the minority class to see if it helps more in a following model. 

In [25]:
#adjust the batch size
model_simpleRNN_updated_2 = Sequential()
model_simpleRNN_updated_2.add(Embedding(input_dim=len(tokenizer_RNN_updated.word_index) + 1, output_dim=128, input_length=30))
model_simpleRNN_updated_2.add(BatchNormalization())
model_simpleRNN_updated_2.add(SimpleRNN(units=50))
model_simpleRNN_updated_2.add(BatchNormalization())
model_simpleRNN_updated_2.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model_simpleRNN_updated_2.add(BatchNormalization())
model_simpleRNN_updated_2.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
model_simpleRNN_updated_2.add(BatchNormalization())
model_simpleRNN_updated_2.add(Dense(2, activation='softmax'))

model_simpleRNN_updated_2.compile(
    loss='sparse_categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy'],    
    weighted_metrics=['accuracy']
)

model_simpleRNN_updated_2.fit(X_train_RNN_updated, y_train_RNN_updated, epochs=100, batch_size=128, validation_split=0.2, sample_weight=sample_weights_updated, callbacks=[reduce_lr])

# test predictions and get evaluation metrics
pred_simpleRNN_updated_2 = model_simpleRNN_updated_2.predict(X_test_RNN_updated)

#convert back
pred_simpleRNN_updated_convert_2 = np.argmax(pred_simpleRNN_updated_2, axis=1)

#evaluate results from the RNN
updated_simpleRNN_class_report_2 = classification_report(y_test_RNN_updated, pred_simpleRNN_updated_convert_2)
updated_simpleRNN_confusion_matrix_class_2 = confusion_matrix(y_test_RNN_updated, pred_simpleRNN_updated_convert_2)
updated_simpleRNN_accuracy_class_2 = accuracy_score(y_test_RNN_updated, pred_simpleRNN_updated_convert_2)

#print the results
print("Classification Report:\n", updated_simpleRNN_class_report_2)
print("Confusion Matrix:\n", updated_simpleRNN_confusion_matrix_class_2)
print("Accuracy:", updated_simpleRNN_accuracy_class_2)

Epoch 1/100
2579/2579 [==============================] - 55s 21ms/step - loss: 0.6478 - accuracy: 0.7265 - weighted_accuracy: 0.7303 - val_loss: 0.5164 - val_accuracy: 0.7791 - val_weighted_accuracy: 0.7525 - lr: 0.0010
Epoch 2/100
2579/2579 [==============================] - 54s 21ms/step - loss: 0.4831 - accuracy: 0.7703 - weighted_accuracy: 0.7725 - val_loss: 0.4929 - val_accuracy: 0.7374 - val_weighted_accuracy: 0.7620 - lr: 0.0010
Epoch 3/100
2579/2579 [==============================] - 55s 21ms/step - loss: 0.4512 - accuracy: 0.7887 - weighted_accuracy: 0.7898 - val_loss: 0.4772 - val_accuracy: 0.7783 - val_weighted_accuracy: 0.7731 - lr: 0.0010
Epoch 4/100
2579/2579 [==============================] - 55s 21ms/step - loss: 0.4333 - accuracy: 0.7996 - weighted_accuracy: 0.8002 - val_loss: 0.4825 - val_accuracy: 0.7873 - val_weighted_accuracy: 0.7725 - lr: 0.0010
Epoch 5/100
2579/2579 [==============================] - 54s 21ms/step - loss: 0.4155 - accuracy: 0.8103 - weighted_accu

Epoch 32/100
2579/2579 [==============================] - 55s 21ms/step - loss: 0.2447 - accuracy: 0.8978 - weighted_accuracy: 0.8986 - val_loss: 0.6538 - val_accuracy: 0.7648 - val_weighted_accuracy: 0.7580 - lr: 1.0000e-05
Epoch 33/100
2579/2579 [==============================] - 55s 21ms/step - loss: 0.2443 - accuracy: 0.8984 - weighted_accuracy: 0.8992 - val_loss: 0.6519 - val_accuracy: 0.7650 - val_weighted_accuracy: 0.7572 - lr: 1.0000e-05
Epoch 34/100
2579/2579 [==============================] - 55s 21ms/step - loss: 0.2441 - accuracy: 0.8984 - weighted_accuracy: 0.8993 - val_loss: 0.6581 - val_accuracy: 0.7650 - val_weighted_accuracy: 0.7578 - lr: 1.0000e-05
Epoch 35/100
2579/2579 [==============================] - 56s 22ms/step - loss: 0.2430 - accuracy: 0.8989 - weighted_accuracy: 0.8998 - val_loss: 0.6562 - val_accuracy: 0.7652 - val_weighted_accuracy: 0.7576 - lr: 1.0000e-05
Epoch 36/100
2579/2579 [==============================] - 55s 21ms/step - loss: 0.2427 - accuracy: 0

2579/2579 [==============================] - 55s 21ms/step - loss: 0.2318 - accuracy: 0.9040 - weighted_accuracy: 0.9050 - val_loss: 0.6929 - val_accuracy: 0.7623 - val_weighted_accuracy: 0.7551 - lr: 1.0000e-05
Epoch 69/100
2579/2579 [==============================] - 55s 21ms/step - loss: 0.2307 - accuracy: 0.9049 - weighted_accuracy: 0.9060 - val_loss: 0.7010 - val_accuracy: 0.7625 - val_weighted_accuracy: 0.7548 - lr: 1.0000e-05
Epoch 70/100
2579/2579 [==============================] - 55s 21ms/step - loss: 0.2314 - accuracy: 0.9044 - weighted_accuracy: 0.9053 - val_loss: 0.6996 - val_accuracy: 0.7630 - val_weighted_accuracy: 0.7543 - lr: 1.0000e-05
Epoch 71/100
2579/2579 [==============================] - 55s 21ms/step - loss: 0.2305 - accuracy: 0.9046 - weighted_accuracy: 0.9057 - val_loss: 0.7005 - val_accuracy: 0.7632 - val_weighted_accuracy: 0.7546 - lr: 1.0000e-05
Epoch 72/100
2579/2579 [==============================] - 55s 21ms/step - loss: 0.2304 - accuracy: 0.9049 - weigh

In [24]:
#these results did not make much of an impact, will now try using smote to oversample the minority class
smote_over = SMOTE(sampling_strategy='auto', random_state=42)
X_train_RNN_over, y_train_RNN_over = smote_over.fit_resample(X_train_RNN_updated, y_train_RNN_updated)

#need to readjust the class weights for this new training data
class_weights_over = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_RNN_over), y=y_train_RNN_over)
class_weights_dict_over = {i: weight for i, weight in enumerate(class_weights_over)}
sample_weights_over = np.array([class_weights_dict_over[y] for y in y_train_RNN_over])

In [25]:
#create the model for the SimpleRNN with smote and complex layers
model_simpleRNN_over = Sequential()
model_simpleRNN_over.add(Embedding(input_dim=len(tokenizer_RNN_updated.word_index) + 1, output_dim=128, input_length=30))
model_simpleRNN_over.add(BatchNormalization())
model_simpleRNN_over.add(SimpleRNN(units=50))
model_simpleRNN_over.add(BatchNormalization())
model_simpleRNN_over.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model_simpleRNN_over.add(BatchNormalization())
model_simpleRNN_over.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
model_simpleRNN_over.add(BatchNormalization())
model_simpleRNN_over.add(Dense(2, activation='softmax'))

model_simpleRNN_over.compile(
    loss='sparse_categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy'],    
    weighted_metrics=['accuracy']
)

#train
model_simpleRNN_over.fit(X_train_RNN_over, y_train_RNN_over, epochs=100, batch_size=64, validation_split=0.2, sample_weight=sample_weights_over, callbacks=[reduce_lr])

Epoch 1/100
6717/6717 [==============================] - 110s 16ms/step - loss: 0.6278 - accuracy: 0.7129 - weighted_accuracy: 0.7129 - val_loss: 0.7732 - val_accuracy: 0.6361 - val_weighted_accuracy: 0.6361 - lr: 0.0010
Epoch 2/100
6717/6717 [==============================] - 110s 16ms/step - loss: 0.4768 - accuracy: 0.7744 - weighted_accuracy: 0.7744 - val_loss: 0.6700 - val_accuracy: 0.6563 - val_weighted_accuracy: 0.6563 - lr: 0.0010
Epoch 3/100
6717/6717 [==============================] - 110s 16ms/step - loss: 0.4621 - accuracy: 0.7822 - weighted_accuracy: 0.7822 - val_loss: 0.9751 - val_accuracy: 0.3273 - val_weighted_accuracy: 0.3273 - lr: 0.0010
Epoch 4/100
6717/6717 [==============================] - 110s 16ms/step - loss: 0.4607 - accuracy: 0.7848 - weighted_accuracy: 0.7848 - val_loss: 0.7974 - val_accuracy: 0.5571 - val_weighted_accuracy: 0.5571 - lr: 0.0010
Epoch 5/100
6717/6717 [==============================] - 110s 16ms/step - loss: 0.4403 - accuracy: 0.7954 - weighted

Epoch 32/100
6717/6717 [==============================] - 111s 17ms/step - loss: 0.3110 - accuracy: 0.8660 - weighted_accuracy: 0.8660 - val_loss: 0.5578 - val_accuracy: 0.7353 - val_weighted_accuracy: 0.7353 - lr: 1.0000e-05
Epoch 33/100
6717/6717 [==============================] - 111s 17ms/step - loss: 0.3108 - accuracy: 0.8667 - weighted_accuracy: 0.8667 - val_loss: 0.5771 - val_accuracy: 0.7272 - val_weighted_accuracy: 0.7272 - lr: 1.0000e-05
Epoch 34/100
6717/6717 [==============================] - 111s 17ms/step - loss: 0.3107 - accuracy: 0.8669 - weighted_accuracy: 0.8669 - val_loss: 0.5808 - val_accuracy: 0.7250 - val_weighted_accuracy: 0.7250 - lr: 1.0000e-05
Epoch 35/100
6717/6717 [==============================] - 111s 17ms/step - loss: 0.3100 - accuracy: 0.8668 - weighted_accuracy: 0.8668 - val_loss: 0.5790 - val_accuracy: 0.7260 - val_weighted_accuracy: 0.7260 - lr: 1.0000e-05
Epoch 36/100
6717/6717 [==============================] - 111s 17ms/step - loss: 0.3101 - accura

6717/6717 [==============================] - 111s 16ms/step - loss: 0.3021 - accuracy: 0.8709 - weighted_accuracy: 0.8709 - val_loss: 0.5829 - val_accuracy: 0.7292 - val_weighted_accuracy: 0.7292 - lr: 1.0000e-05
Epoch 69/100
6717/6717 [==============================] - 111s 17ms/step - loss: 0.3019 - accuracy: 0.8707 - weighted_accuracy: 0.8707 - val_loss: 0.5844 - val_accuracy: 0.7268 - val_weighted_accuracy: 0.7268 - lr: 1.0000e-05
Epoch 70/100
6717/6717 [==============================] - 111s 17ms/step - loss: 0.3015 - accuracy: 0.8711 - weighted_accuracy: 0.8711 - val_loss: 0.5792 - val_accuracy: 0.7338 - val_weighted_accuracy: 0.7338 - lr: 1.0000e-05
Epoch 71/100
6717/6717 [==============================] - 111s 17ms/step - loss: 0.3014 - accuracy: 0.8712 - weighted_accuracy: 0.8712 - val_loss: 0.5844 - val_accuracy: 0.7293 - val_weighted_accuracy: 0.7293 - lr: 1.0000e-05
Epoch 72/100
6717/6717 [==============================] - 111s 17ms/step - loss: 0.3007 - accuracy: 0.8717 - 

In [26]:
# test predictions and get evaluation metrics
pred_simpleRNN_over = model_simpleRNN_over.predict(X_test_RNN_updated)

#convert back
pred_simpleRNN_over_convert = np.argmax(pred_simpleRNN_over, axis=1)

#evaluate results from the RNN
over_simpleRNN_class_report = classification_report(y_test_RNN_updated, pred_simpleRNN_over_convert)
over_simpleRNN_confusion_matrix_class = confusion_matrix(y_test_RNN_updated, pred_simpleRNN_over_convert)
over_simpleRNN_accuracy_class = accuracy_score(y_test_RNN_updated, pred_simpleRNN_over_convert)

#print the results
print("Classification Report:\n", over_simpleRNN_class_report)
print("Confusion Matrix:\n", over_simpleRNN_confusion_matrix_class)
print("Accuracy:", over_simpleRNN_accuracy_class)

3224/3224 [==============================] - 3s 883us/step
Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.66      0.68     36165
           1       0.82      0.84      0.83     66983

    accuracy                           0.78    103148
   macro avg       0.76      0.75      0.76    103148
weighted avg       0.78      0.78      0.78    103148

Confusion Matrix:
 [[23908 12257]
 [10397 56586]]
Accuracy: 0.780373831775701


## Part 8: Scraping Reviews to Get New Data From a Website

In [27]:
#ensure libraries are installed for scraping needs [20][21]
!pip install beautifulsoup4
!pip install --upgrade selenium

In [28]:
# import relevant libraries
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import os

In [29]:
# next, I need to specify the driver path for the web driver for Selenium to work
#set up selenium webdriver [21]
service = Service('/Users/coreyreid/Documents/Final Project/Final Project - Implementation & Evaluation/chromedriver')
options = webdriver.ChromeOptions()
#options.add_argument('--headless') #to make it headless and not run the chrome instance
driver = webdriver.Chrome(service=service, options=options)

#example website to use for the scraping
url = "https://www.mainecottagekeepers.com/2-beautiful-water-view-properties-in-1-for-extended-families-orp5b4a2abx"
driver.get(url)

# delay as needed
time.sleep(10)  

# Switch to the iframe containing the reviews - need to switch the focus into the iframe
iframe = driver.find_element(By.CSS_SELECTOR, 'div.ownerrez-widget[data-widgetid="dc0c1ee92566447e81ba5e68a8a2ac1a"] iframe')
driver.switch_to.frame(iframe)

# delay as needed
time.sleep(10)  

# Extract the HTML content
html = driver.page_source

# now, I will scrape the website to expose the review content
soup = BeautifulSoup(html, 'html.parser')
review_elements = soup.find_all('div', class_='review-item')

In [30]:
#now that we have isolated the html element that matters, we will want to go through all the reviews
#and collect just the relevant text
#initialize
real_reviews = []

#loop through the review elements
for review in review_elements:
    # Scrape the review title text
    review_title = review.find("span", class_="review-item-title").strong
    # Scrape the review body text
    review_body = review.find("div", class_="has-read-more").text.strip()
    #check that a review exists
    if review_title and review_body:
        #get review title text 
        review_title_text = review_title.text.strip()
        #get review body text
        review_body_text = review_body
        # put the data in a dataframe
                # Append the scraped data as a dictionary to the 'data' list
        real_reviews.append(
            review_body_text
        )
        #print the details through the loop
        print("The title of the review is:", review_title_text)
        print("The review content is:", review_body_text)
        print("---------------------------------")
    else:
        #error handling
        print("There is no review title or text.")

The title of the review is: 
The review content is: A big thank you from us to be able to stay in this wonderful place in order to be with our family during the Lobstah fest.  We drove from Florida to see our Massachusetts family as we gathered in Maine to welcome our Grandson from the ship that anchored in Rockland for the Festival.  We loved being able to see him as he did us.  We felt at home in this Airbnb as we gathered at the fire pit and the deck table for meals. We did several meals at the Dip Net too.  Great food and very close to house.This house is very well equipped, clean, and is an old farmhouse with a lot of Character.We did host an Airbnb on the Cape at one time and know that Shawn-Elise deserves a 5 star rating to become a super host if she is not already.Would we come again?  You Bet.Sharyn
---------------------------------
The title of the review is: 
The review content is: Great stay. Just didn’t understand why they took 500 for a security deposit
------------------

In [31]:
# test the way the data is structured by looking at the output
real_reviews

['A big thank you from us to be able to stay in this wonderful place in order to be with our family during the Lobstah fest.\xa0 We drove from Florida to see our Massachusetts family as we gathered in Maine to welcome our Grandson from the ship that anchored in Rockland for the Festival.\xa0 We loved being able to see him as he did us.\xa0 We felt at home in this Airbnb as we gathered at the fire pit and the deck table for meals. We did several meals at the Dip Net too.\xa0 Great food and very close to house.This house is very well equipped, clean, and is an old farmhouse with a lot of Character.We did host an Airbnb on the Cape at one time and know that Shawn-Elise deserves a 5 star rating to become a super host if she is not already.Would we come again?\xa0 You Bet.Sharyn',
 'Great stay. Just didn’t understand why they took 500 for a security deposit',
 'The place is absolutely beautiful and there was plenty of room for all 10 f us to sleep. The houses are just a short drive from a b

## Part 9: Testing on Real Review Data

In [33]:
# in order to scale the preprocessing to be able to be used with new reviews, I need to generalize the 
# preprocessing function - I will do that now for the one with no POS tagging

def preprocess_no_pos(review):
    
    #remove special characters and ensure lowercase
    def preprocess_remove_special(review):
        review_handled = re.sub(r'[^a-zA-Z\s]', '', review)
        review_lower = review_handled.lower()
        return review_lower

    #tokenize
    def preprocess_tokenize(review):
        bag_of_words = word_tokenize(review)
        return bag_of_words

    #remove stopwords
    def preprocess_stopwords(review):
        words = stopwords.words('english')
        set_stop_words = set(words)
        removed_stopwords_words = []
        for word in review:
            if word not in set_stop_words:
                removed_stopwords_words.append(word)
        return removed_stopwords_words

    #lemmatize
    def preprocess_lemmatize(review):
        lemmatizer = WordNetLemmatizer()
        lemmatized_review = [lemmatizer.lemmatize(word) for word in review ]
        return lemmatized_review

    # Apply the preprocessing steps
    review = preprocess_remove_special(review)
    review = preprocess_tokenize(review)
    review = preprocess_stopwords(review)
    review = preprocess_lemmatize(review)

    # Join the tokens back into a single string
    preprocessed_review = ' '.join(review)

    #return the preprocessed reviews
    return preprocessed_review

In [34]:
# turns out they are all positive, which when you look above you can see is the case - this will happen 
# from time to time. I will manually add data to the reviews data to put in a few negative reviews for further 
# testing - this will confirm the model is working properly

updated_real_reviews = real_reviews.copy()

updated_real_reviews.extend(['Awful', 'It was a bad rental unit and was very ugly. The ammenities were all broken.', 'I will never return to this or recommend it to anyone - it was a horrible rental and it ruined my holiday.'])

updated_real_reviews

['A big thank you from us to be able to stay in this wonderful place in order to be with our family during the Lobstah fest.\xa0 We drove from Florida to see our Massachusetts family as we gathered in Maine to welcome our Grandson from the ship that anchored in Rockland for the Festival.\xa0 We loved being able to see him as he did us.\xa0 We felt at home in this Airbnb as we gathered at the fire pit and the deck table for meals. We did several meals at the Dip Net too.\xa0 Great food and very close to house.This house is very well equipped, clean, and is an old farmhouse with a lot of Character.We did host an Airbnb on the Cape at one time and know that Shawn-Elise deserves a 5 star rating to become a super host if she is not already.Would we come again?\xa0 You Bet.Sharyn',
 'Great stay. Just didn’t understand why they took 500 for a security deposit',
 'The place is absolutely beautiful and there was plenty of room for all 10 f us to sleep. The houses are just a short drive from a b

In [35]:
# next, I will test this set with Multinomial Naive Bayes with no POS tags
# to start I will preprocess the data using this function and looping through all the reviews
updated_real_reviews_processed = [preprocess_no_pos(review) for review in updated_real_reviews]

In [38]:
#last I will test the best 3-class RNN model with the real review data
# to do this, we will need to process the data a bit more to get it into it's final model training state.
# we will use the no POS tagging version as it always seemed to perform a bit better, even if the difference was
# very small between no POS tags and POS tagged data

# prepare the review text data for RNN usage
tokenizer_RNN_noPOS_test_3 = Tokenizer()
tokenizer_RNN_noPOS_test_3.fit_on_texts(updated_real_reviews_processed)
review_prepped_RNN_noPOS_test = tokenizer_RNN_noPOS_test_3.texts_to_sequences(updated_real_reviews_processed)

# pad the text with a set length - used 10 previously so will use it again
review_prepped_RNN_noPOS_padded = pad_sequences(review_prepped_RNN_noPOS_test, maxlen=30)

real_pred_RNN_noPOS_test = model_simpleRNN_normalized_only.predict(review_prepped_RNN_noPOS_padded)

# convert back to positive, negative, or neutral
real_pred_RNN_noPOS_converted_back_test = label_encoder_RNN_noPOS.inverse_transform_sentiment_noPOS(real_pred_RNN_noPOS_test.argmax(axis=1))

# get the predictions
real_pred_RNN_noPOS_converted_back_test

1/1 [==============================] - 0s 64ms/step


['positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive',
 'positive']

As can be seen, the 3-class predictor is overfit to positive and chooses positive each time.

In [47]:
# additionally, I will test the sentiment analysis on the redefined threshold 2-class  models 
# which performed a bit better than the previous 3-class's best results


# prepare the review text data for RNN usage
tokenizer_RNN_noPOS_test_2 = Tokenizer()
tokenizer_RNN_noPOS_test_2.fit_on_texts(updated_real_reviews_processed)
review_prepped_RNN_noPOS_test_2 = tokenizer_RNN_noPOS_test_2.texts_to_sequences(updated_real_reviews_processed)

# pad the text with a set length - used 10 previously so will use it again
review_prepped_RNN_noPOS_padded_2 = pad_sequences(review_prepped_RNN_noPOS_test_2, maxlen=30)

real_pred_RNN_noPOS_test_2 = model_simpleRNN_over.predict(review_prepped_RNN_noPOS_padded_2)

encoded_labels = real_pred_RNN_noPOS_test_2.argmax(axis=1)

# convert back to positive, negative, or neutral
real_pred_RNN_noPOS_converted_back_test_2 = LabelEncoderSentiment_updated.inverse_transform_sentiment_updated(encoded_labels)

# get the predictions
real_pred_RNN_noPOS_converted_back_test_2

1/1 [==============================] - 0s 8ms/step


['negative',
 'positive',
 'positive',
 'negative',
 'negative',
 'positive',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative']

## Part 10: Product Development

I will start by building a product for the 3-class model

In [48]:
#Import the necessary GUI libraries
#I will use message box from Tkinter to build the GUI and the Tkinter library in general [22]

import tkinter as tk
from tkinter import messagebox
import joblib
import string

In [57]:
#first I need to define a function that will run on the button click. 
# this will include the preprocessing, vectorization, and model execution for the sentiment analysis

# function to determine the preprocessed state for the reviews
def determine_preprocess(review):
    # Create a list of reviews, for any ";" will seperate the list so input needs to seperate reviews by ";"
    reviews_list = review.split(";")
    #initialize the results list to be used to append the results
    reviews_list_processed = []
    
    #loop through the reviews and analyze individually
    for review in reviews_list:
        # Preprocess each review individually
        processed_review = preprocess_no_pos(review) 
        tokenizer_RNN_noPOS_prod = Tokenizer()
        tokenizer_RNN_noPOS_prod.fit_on_texts(processed_review)
        review_prepped_RNN_noPOS_prod = tokenizer_RNN_noPOS_prod.texts_to_sequences(processed_review)
        # pad the text with a set length - used 10 previously so will use it again
        review_prepped_RNN_noPOS_padded_prod = pad_sequences(review_prepped_RNN_noPOS_prod, maxlen=30)
        # predict the sentiment of the reviews
        reviews_list_predictions = model_simpleRNN_normalized_only.predict(review_prepped_RNN_noPOS_padded_prod)
        # convert back to positive, negative, or neutral
        sentiment_labels = label_encoder_RNN_noPOS.inverse_transform_sentiment_noPOS(real_pred_RNN_noPOS_test.argmax(axis=1))
        #append the results
        reviews_list_processed.append((review, sentiment_labels[0])) 
    #return the results
    return reviews_list_processed

# create a function for the sentiment analysis
def get_sentiment():
    #get the entry from the form 
    user_reviews_list = entry.get()
    # if there are reviews, then analyze them 
    if user_reviews_list:
        #determine sentiment
        sentiment_out = determine_preprocess(user_reviews_list)
        #clear the previous review info to ensure each review is looked at individually
        review_text.delete(1.0, tk.END)
        
        # analyse the review
        for review, sentiment in sentiment_out:
            # in the text output that is part of the GUI, we will now get the values needed in our presentation of results
            # get the review
            review_text.insert(tk.END, "Review:\n{}\n".format(review))
            # get the sentiment
            review_text.insert(tk.END, "Sentiment: {}\n".format(sentiment))
            
            #logic for giving recommendations on the handling procedures for the review
            #positive
            if sentiment == 'positive':
                message = "Message: This is a positive review. You should consider reaching out to learn more about what was so great about the stay and thank them for leaving a good review."
            #negative
            elif sentiment == 'negative':
                message = "Message: This is a negative review. You should consider reaching out to learn more about their bad experience and offer them a 10% discount for their next stay. Also thank them for the feedback."
            #nuetral or other
            else:
                message = "Message: Sentiment is uncertain. You may want to connect with them for more information."
            
            #add the message to the text output for presentation
            review_text.insert(tk.END, message + "\n\n")
    #if no reviews input and button clicked
    else:
        #give an error to the user
        messagebox.showerror("Error", "Please enter a review.")


# initialize the tkinter model
root = tk.Tk()
#provide a title for the GUI
root.title("Short-Term Rental Review Sentiment Analysis")

# give the label for the GUI input
label = tk.Label(root, text="Enter your Short-Term Rental review (seperate multiple reviews with a ';')")
#update the label
label.pack()

#define the form entry size
entry = tk.Entry(root, width=50)
#update the entry
entry.pack()

#define the button and actions from the button click - in this case we will run the get_sentiment function
analyze_button = tk.Button(root, text="Determine Sentiment", command=get_sentiment)
#update the button
analyze_button.pack()

# Size the text widget which will be used to display the review analysis output - including the resolution message 
review_text = tk.Text(root, height=10, width=50)
# update the text details
review_text.pack()

#loop the root GUI to keep the window running
root.mainloop()

1/1 [==============================] - 0s 13ms/step


As can be seen when you run the code, a GUI is created that allows the user to input reviews - multiple are seperated with a ";" - and get outcomes from the sentiment analysis with recommendations on what to do. This proves that a sentiment analysis tool like this could be created, and the minimal viable product is completed. 

Next, I will build a 2-class model GUI for demonstrative purposes

In [60]:
#first I need to define a function that will run on the button click. 
# this will include the preprocessing, vectorization, and model execution for the sentiment analysis

# function to determine the preprocessed state for the reviews
def determine_preprocess_2(review):
    # Create a list of reviews, for any ";" will seperate the list so input needs to seperate reviews by ";"
    reviews_list_2 = review.split(";")
    #initialize the results list to be used to append the results
    reviews_list_processed_2 = []
    
    #loop through the reviews and analyze individually
    for review in reviews_list_2:
        # Preprocess each review individually
        processed_review_2 = preprocess_no_pos(review) 
        tokenizer_RNN_noPOS_prod_2 = Tokenizer()
        tokenizer_RNN_noPOS_prod_2.fit_on_texts(processed_review_2)
        review_prepped_RNN_noPOS_prod_2 = tokenizer_RNN_noPOS_test_2.texts_to_sequences(processed_review_2)
        # pad the text with a set length - used 10 previously so will use it again
        review_prepped_RNN_noPOS_padded_prod_2 = pad_sequences(review_prepped_RNN_noPOS_prod_2, maxlen=30)
        # predict the sentiment of the reviews
        reviews_list_predictions_2 = model_simpleRNN_over.predict(review_prepped_RNN_noPOS_padded_prod_2)
        # convert back to positive, negative, or neutral
        sentiment_labels_2 = label_encoder_RNN_noPOS.inverse_transform_sentiment_noPOS(real_pred_RNN_noPOS_test_2.argmax(axis=1))
        #append the results
        reviews_list_processed_2.append((review, sentiment_labels_2[0])) 
    #return the results
    return reviews_list_processed_2

# create a function for the sentiment analysis
def get_sentiment_2():
    #get the entry from the form 
    user_reviews_list_2 = entry_2.get()
    # if there are reviews, then analyze them 
    if user_reviews_list_2:
        #determine sentiment
        sentiment_out_2 = determine_preprocess_2(user_reviews_list_2)
        #clear the previous review info to ensure each review is looked at individually
        review_text.delete(1.0, tk.END)
        
        # analyse the review
        for review, sentiment in sentiment_out_2:
            # in the text output that is part of the GUI, we will now get the values needed in our presentation of results
            # get the review
            review_text.insert(tk.END, "Review:\n{}\n".format(review))
            # get the sentiment
            review_text.insert(tk.END, "Sentiment: {}\n".format(sentiment))
            
            #logic for giving recommendations on the handling procedures for the review
            #positive
            if sentiment == 'positive':
                message = "Message: This is a positive review. You should consider reaching out to learn more about what was so great about the stay and thank them for leaving a good review."
            #negative
            elif sentiment == 'negative':
                message = "Message: This is a negative review. You should consider reaching out to learn more about their bad experience and offer them a 10% discount for their next stay. Also thank them for the feedback."

            #add the message to the text output for presentation
            review_text.insert(tk.END, message + "\n\n")
    #if no reviews input and button clicked
    else:
        #give an error to the user
        messagebox.showerror("Error", "Please enter a review.")


# initialize the tkinter model
root_2 = tk.Tk()
#provide a title for the GUI
root_2.title("Short-Term Rental Review Sentiment Analysis")

# give the label for the GUI input
label_2 = tk.Label(root_2, text="Enter your Short-Term Rental review (seperate multiple reviews with a ';')")
#update the label
label_2.pack()

#define the form entry size
entry_2 = tk.Entry(root_2, width=50)
#update the entry
entry_2.pack()

#define the button and actions from the button click - in this case we will run the get_sentiment function
analyze_button_2 = tk.Button(root_2, text="Determine Sentiment", command=get_sentiment_2)
#update the button
analyze_button_2.pack()

# Size the text widget which will be used to display the review analysis output - including the resolution message 
review_text = tk.Text(root_2, height=10, width=50)
# update the text details
review_text.pack()

#loop the root GUI to keep the window running
root_2.mainloop()

2/2 [==============================] - 0s 2ms/step


While the 2-class predictor performed better, it was still not as accurate as the Random Forest model. 